In [1329]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import time

from datetime import datetime
pd.set_option('display.max_columns', 100)
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
from pprint import pprint
from sklearn import preprocessing   
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
import  seaborn as sns
import pytz
from datetime import datetime
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [1331]:
# assuming now contains a timezone aware datetime
now = datetime.now()
tz = pytz.timezone('Asia/Kolkata')
your_now = now.astimezone(tz)
print (your_now)

2020-01-26 15:28:48.063426+05:30


In [0]:
df_train = pd.read_csv('/content/drive/My Drive/Hackathon/LTFS/25-JAN/Train_set - Segment_2.csv')
# /content/drive/My Drive/Hackathon/LTFS/25-JAN/Train_set - Segment_2.csv

df_test = pd.read_csv('/content/drive/My Drive/Hackathon/LTFS/25-JAN/Test_Segement_2_Consolidate_1488.csv')
# /content/drive/My Drive/Hackathon/LTFS/25-JAN/Test_Segement_2_Consolidate_1488.csv

df_holiday = pd.read_csv('/content/drive/My Drive/Hackathon/LTFS/25-JAN/Data/Holidays.csv')

df_dt = pd.read_csv('/content/drive/My Drive/Hackathon/LTFS/25-JAN/Data/Dates.csv')

In [1333]:
print('df_train -', df_train.shape)
print('df_test -', df_test.shape)
print('df_dt -', df_dt.shape)
print('df_holiday -', df_holiday.shape)


df_train - (13504, 6)
df_test - (1488, 4)
df_dt - (944, 2)
df_holiday - (68, 3)


In [1334]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13504 entries, 0 to 13503
Data columns (total 6 columns):
application_date    13504 non-null object
segment             13504 non-null int64
branch_id           0 non-null float64
state               13504 non-null object
zone                0 non-null float64
case_count          13504 non-null int64
dtypes: float64(2), int64(2), object(2)
memory usage: 633.1+ KB


In [1335]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
segment,13504.0,2.000000,0.000000,2.0,2.0,2.0,2.0,2.0
branch_id,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zone,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
case_count,13504.0,942.318646,1624.339623,0.0,54.0,371.0,1033.0,13787.0


In [1336]:
df_train.columns

Index(['application_date', 'segment', 'branch_id', 'state', 'zone',
       'case_count'],
      dtype='object')

In [0]:
df_train_bkp = df_train.copy(deep=True)

In [0]:
df_train['application_date'] = df_train['application_date'].astype('datetime64[ns]')

In [0]:
df_train['application_date'] = df_train.application_date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [1340]:
df_train.sample(5)

,application_date,segment,branch_id,state,zone,case_count
3318,05/27/2019,2,NaN,GUJARAT,NaN,428
988,08/23/2017,2,NaN,BIHAR,NaN,43
7828,11/19/2017,2,NaN,MADHYA PRADESH,NaN,579
13411,04/22/2019,2,NaN,WEST BENGAL,NaN,2305
5754,02/20/2019,2,NaN,KERALA,NaN,1431


In [1341]:
df_holiday.head()

,H_date,Holiday_Flag,Holiday_name
0,1-Jan-17,1,New Years Day
1,14-Jan-17,1,Makar Sankranti / Pongal
2,26-Jan-17,1,Republic Day
3,24-Feb-17,1,Maha Shivaratri
4,13-Mar-17,1,Holi


In [0]:
df_train_nw = df_train.copy(deep=True)

In [1343]:
df_train_nw.shape

(13504, 6)

In [1344]:
df_train_nw.sample(5)

,application_date,segment,branch_id,state,zone,case_count
8338,04/13/2019,2,NaN,MADHYA PRADESH,NaN,1401
1049,10/23/2017,2,NaN,BIHAR,NaN,195
7002,12/07/2017,2,NaN,MAHARASHTRA,NaN,300
7762,09/14/2017,2,NaN,MADHYA PRADESH,NaN,1076
6500,11/14/2018,2,NaN,KARNATAKA,NaN,3619


In [0]:
df_train_nw.drop(columns='segment',inplace=True)

In [0]:
df_train_nw.drop(columns=['branch_id','zone'],inplace=True)

In [1347]:
df_train_nw.shape

(13504, 3)

In [1348]:
df_train_nw.sample(4)

,application_date,state,case_count
3277,04/16/2019,GUJARAT,607
6008,07/10/2017,KARNATAKA,1444
5706,01/03/2019,KERALA,140
8821,04/17/2018,ORISSA,3946


In [0]:
#################################################################  *********  ADDING HOLIDAYS ********* ####################################################################################

In [1350]:
df_holiday.head()

,H_date,Holiday_Flag,Holiday_name
0,1-Jan-17,1,New Years Day
1,14-Jan-17,1,Makar Sankranti / Pongal
2,26-Jan-17,1,Republic Day
3,24-Feb-17,1,Maha Shivaratri
4,13-Mar-17,1,Holi


In [0]:
df_holiday['H_date'] = df_holiday['H_date'].astype('datetime64[ns]')

In [1352]:
df_holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 3 columns):
H_date          68 non-null datetime64[ns]
Holiday_Flag    68 non-null int64
Holiday_name    68 non-null object
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.7+ KB


In [0]:
df_holiday.dropna(axis=0,inplace=True)

In [0]:
df_holiday['H_date'] = df_holiday.H_date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [0]:
df_train_h = pd.merge(df_train_nw,df_holiday,how='left',left_on='application_date',right_on='H_date')

In [1356]:
df_train_h.head(200)

,application_date,state,case_count,H_date,Holiday_Flag,Holiday_name
0,04/01/2017,ASSAM,0,NaN,NaN,NaN
1,04/02/2017,ASSAM,0,NaN,NaN,NaN
2,04/03/2017,ASSAM,0,NaN,NaN,NaN
3,04/04/2017,ASSAM,0,NaN,NaN,NaN
4,04/05/2017,ASSAM,0,04/05/2017,1.0,Ram Navami
...,...,...,...,...,...,...
195,10/13/2017,ASSAM,464,NaN,NaN,NaN
196,10/14/2017,ASSAM,357,NaN,NaN,NaN
197,10/15/2017,ASSAM,92,NaN,NaN,NaN
198,10/16/2017,ASSAM,500,NaN,NaN,NaN


In [0]:
df_train_h.drop(columns=['H_date','Holiday_name'],inplace=True)

In [0]:
df_train_h['Holiday_Flag'].replace(np.NaN,0,inplace=True)

In [1359]:
df_train_h['Holiday_Flag'].unique()

array([0., 1.])

In [1360]:
df_train_h.head(200)

,application_date,state,case_count,Holiday_Flag
0,04/01/2017,ASSAM,0,0.0
1,04/02/2017,ASSAM,0,0.0
2,04/03/2017,ASSAM,0,0.0
3,04/04/2017,ASSAM,0,0.0
4,04/05/2017,ASSAM,0,1.0
...,...,...,...,...
195,10/13/2017,ASSAM,464,0.0
196,10/14/2017,ASSAM,357,0.0
197,10/15/2017,ASSAM,92,0.0
198,10/16/2017,ASSAM,500,0.0


In [1361]:
df_train_h.sample(5)

,application_date,state,case_count,Holiday_Flag
2979,06/22/2018,GUJARAT,742,0.0
6384,07/21/2018,KARNATAKA,4160,0.0
7848,12/09/2017,MADHYA PRADESH,432,0.0
11310,03/05/2018,TRIPURA,101,0.0
9981,02/27/2019,PUNJAB,0,0.0


In [1362]:
df_train_h['application_date'] = df_train_h['application_date'].astype('datetime64[ns]')
df_train_h.sample(5)

,application_date,state,case_count,Holiday_Flag
11434,2018-07-07,TRIPURA,216,0.0
12649,2019-07-13,UTTAR PRADESH,606,0.0
12543,2019-03-29,UTTAR PRADESH,575,0.0
9170,2019-04-01,ORISSA,290,0.0
13117,2018-07-02,WEST BENGAL,274,0.0


In [1363]:
df_train_h.set_index('application_date',inplace=True)
df_train_h.sample(5)

,state,case_count,Holiday_Flag
application_date,,,
2018-05-22,UTTAR PRADESH,400,0.0
2019-04-04,UTTAR PRADESH,109,0.0
2018-10-27,TAMIL NADU,9392,0.0
2017-07-04,MADHYA PRADESH,186,0.0
2019-05-13,MAHARASHTRA,513,0.0


In [1364]:
# Add columns with year, month, and weekday name
df_train_h['day'] = df_train_h.index.day
df_train_h['Year'] = df_train_h.index.year
df_train_h['Month'] = df_train_h.index.month
df_train_h['Weekday_Name'] = df_train_h.index.weekday_name
# Display a random sampling of 5 rows
df_train_h.sample(5, random_state=0)

,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name
application_date,,,,,,,
2018-11-05,BIHAR,542,0.0,5,2018,11,Monday
2018-01-28,TRIPURA,16,0.0,28,2018,1,Sunday
2017-04-21,KARNATAKA,906,0.0,21,2017,4,Friday
2017-11-07,UTTAR PRADESH,107,0.0,7,2017,11,Tuesday
2019-05-24,HARYANA,0,0.0,24,2019,5,Friday


In [0]:
df_train_h.reset_index(inplace=True)

In [1366]:
df_train_h['application_date'] = df_train_h.application_date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))
df_train_h.sample(5)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name
840,07/20/2019,ASSAM,986,0.0,20,2019,7,Saturday
731,04/02/2019,ASSAM,7,0.0,2,2019,4,Tuesday
3876,08/14/2018,HARYANA,0,0.0,14,2018,8,Tuesday
1488,01/05/2019,BIHAR,296,0.0,5,2019,1,Saturday
3423,05/18/2017,HARYANA,0,0.0,18,2017,5,Thursday


In [0]:
######################################################## Label ENCODING #############################################

In [0]:
label_encoder = preprocessing.LabelEncoder() 

In [0]:
#df_train_h['branch_id'] = label_encoder.fit_transform(df_train_h['branch_id'])
df_train_h['Holiday_Flag'] = label_encoder.fit_transform(df_train_h['Holiday_Flag'])
df_train_h['Weekday_Name'] = label_encoder.fit_transform(df_train_h['Weekday_Name'])
df_train_h['state'] = label_encoder.fit_transform(df_train_h['state'])
#df_train_h['zone'] = label_encoder.fit_transform(df_train_h['zone'])

In [1370]:
df_train_h.sample(5)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name
3392,04/17/2017,4,0,0,17,2017,4,1
5684,12/12/2018,7,1505,0,12,2018,12,6
10803,02/05/2019,12,1547,0,5,2019,2,5
10359,11/18/2017,12,8246,0,18,2017,11,2
1572,03/30/2019,1,2526,0,30,2019,3,2


In [0]:
######################################################## One HOT ENCODING #############################################

In [0]:
#df_train_h_dummy = pd.concat([df_train_h,pd.get_dummies(df_train_h['branch_id'], prefix='BR',dummy_na=False)],axis=1)
#df_train_h_dummy = pd.concat([df_train_h_dummy,pd.get_dummies(df_train_h['state'], prefix='ST',dummy_na=False)],axis=1)
#df_train_h_dummy = pd.concat([df_train_h_dummy,pd.get_dummies(df_train_h['zone'], prefix='ZN',dummy_na=False)],axis=1)
#df_train_h_dummy = pd.concat([df_train_h_dummy,pd.get_dummies(df_train_h['Holiday_name'], prefix='HD',dummy_na=False)],axis=1)
#df_train_h_dummy = pd.concat([df_train_h_dummy,pd.get_dummies(df_train_h['day'], prefix='DAY',dummy_na=False)],axis=1)
#df_train_h_dummy = pd.concat([df_train_h_dummy,pd.get_dummies(df_train_h['Year'], prefix='YR',dummy_na=False)],axis=1)
#df_train_h_dummy = pd.concat([df_train_h_dummy,pd.get_dummies(df_train_h['Month'], prefix='MTH',dummy_na=False)],axis=1)
#df_train_h_dummy = pd.concat([df_train_h_dummy,pd.get_dummies(df_train_h['Weekday_Name'], prefix='WD',dummy_na=False)],axis=1)

In [0]:
# df_train_h_dummy.sample(5, random_state=0)

In [0]:
# df_train_nw_case_count_datewise['Weekday_Name'] = label_encoder.fit_transform(df_train_nw_case_count_datewise['Weekday_Name'])
# df_train_nw_case_count_datewise['Weekday_Name'].unique()

In [0]:
#df_train_h_dummy.drop(columns=['branch_id','state','zone','Holiday_name','day','Year','Month','Weekday_Name'],inplace=True)

In [0]:
#df_train_h_dummy.sample(5, random_state=0)

In [0]:
# df_train_h_dummy.shape

In [0]:
df_train_h_dummy = df_train_h.copy(deep=True)

In [1379]:
df_train_h_dummy.shape

(13504, 8)

In [0]:
#################################################### Building the Test Segment -1  ###################################################

In [0]:
# df_test = df_test_bkp.copy(deep=True)

In [1382]:
df_test.head()

,Unnamed: 0,Unnamed: 0.1,application_date,state
0,0,0,7/24/2019,ASSAM
1,1,1,7/25/2019,ASSAM
2,2,2,7/26/2019,ASSAM
3,3,3,7/27/2019,ASSAM
4,4,4,7/28/2019,ASSAM


In [0]:
df_test.drop(columns=['Unnamed: 0','Unnamed: 0.1'],inplace=True)

In [0]:
df_test_bkp = df_test.copy(deep=True)

In [0]:
# df_test.drop(columns='id',inplace=True)

In [0]:
# df_test.drop(columns='segment',inplace=True)

In [1387]:
df_test.head()

,application_date,state
0,7/24/2019,ASSAM
1,7/25/2019,ASSAM
2,7/26/2019,ASSAM
3,7/27/2019,ASSAM
4,7/28/2019,ASSAM


In [0]:
df_test['application_date'] = df_test['application_date'].astype('datetime64[ns]')

In [0]:
df_test['application_date'] = df_test.application_date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [1390]:
df_test.shape

(1488, 2)

In [1391]:
df_holiday.head()

,H_date,Holiday_Flag,Holiday_name
0,01/01/2017,1,New Years Day
1,01/14/2017,1,Makar Sankranti / Pongal
2,01/26/2017,1,Republic Day
3,02/24/2017,1,Maha Shivaratri
4,03/13/2017,1,Holi


In [0]:
df_holiday['H_date'] = df_holiday['H_date'].astype('datetime64[ns]')

In [1393]:
df_holiday.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 0 to 67
Data columns (total 3 columns):
H_date          68 non-null datetime64[ns]
Holiday_Flag    68 non-null int64
Holiday_name    68 non-null object
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.1+ KB


In [0]:
df_holiday.dropna(axis=0,inplace=True)

In [0]:
df_holiday['H_date'] = df_holiday.H_date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [1396]:
df_holiday.head()

,H_date,Holiday_Flag,Holiday_name
0,01/01/2017,1,New Years Day
1,01/14/2017,1,Makar Sankranti / Pongal
2,01/26/2017,1,Republic Day
3,02/24/2017,1,Maha Shivaratri
4,03/13/2017,1,Holi


In [1397]:
df_holiday.shape

(68, 3)

In [0]:
df_test_h = pd.merge(df_test,df_holiday,how='left',left_on='application_date',right_on='H_date')

In [1399]:
df_test_h.shape

(1488, 5)

In [1400]:
df_test_h.head(6)

,application_date,state,H_date,Holiday_Flag,Holiday_name
0,07/24/2019,ASSAM,NaN,NaN,NaN
1,07/25/2019,ASSAM,NaN,NaN,NaN
2,07/26/2019,ASSAM,NaN,NaN,NaN
3,07/27/2019,ASSAM,NaN,NaN,NaN
4,07/28/2019,ASSAM,NaN,NaN,NaN
5,07/29/2019,ASSAM,NaN,NaN,NaN


In [1401]:
df_test_h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1488 entries, 0 to 1487
Data columns (total 5 columns):
application_date    1488 non-null object
state               1488 non-null object
H_date              128 non-null object
Holiday_Flag        128 non-null float64
Holiday_name        128 non-null object
dtypes: float64(1), object(4)
memory usage: 69.8+ KB


In [0]:
df_test_h.drop(columns=['H_date','Holiday_name'],inplace=True)

In [0]:
df_test_h['Holiday_Flag'].replace(np.NaN,0,inplace=True)

In [0]:
#df_test_h['Holiday_Flag'] = df_test_h['Holiday_Flag'].astype('int64')

In [1405]:
df_test_h.head(5)

,application_date,state,Holiday_Flag
0,07/24/2019,ASSAM,0.0
1,07/25/2019,ASSAM,0.0
2,07/26/2019,ASSAM,0.0
3,07/27/2019,ASSAM,0.0
4,07/28/2019,ASSAM,0.0


In [0]:
# df_test_h['Holiday_Flag'].nunique()

In [1407]:
df_test_h['application_date'] = df_test_h['application_date'].astype('datetime64[ns]')
df_test_h.sample(5)

,application_date,state,Holiday_Flag
1043,2019-08-13,PUNJAB,0.0
1323,2019-08-14,UTTAR PRADESH,0.0
169,2019-10-08,BIHAR,1.0
627,2019-10-01,KERALA,0.0
798,2019-09-16,MAHARASHTRA,0.0


In [1408]:
df_test_h.set_index('application_date',inplace=True)
df_test_h.sample(5)

,state,Holiday_Flag
application_date,,
2019-09-14,PUNJAB,0.0
2019-08-07,BIHAR,0.0
2019-08-03,TRIPURA,0.0
2019-10-04,ASSAM,0.0
2019-09-09,WEST BENGAL,0.0


In [1409]:
# Add columns with year, month, and weekday name
df_test_h['day'] = df_test_h.index.day
df_test_h['Year'] = df_test_h.index.year
df_test_h['Month'] = df_test_h.index.month
df_test_h['Weekday_Name'] = df_test_h.index.weekday_name
# Display a random sampling of 5 rows
df_test_h.sample(5, random_state=0)

,state,Holiday_Flag,day,Year,Month,Weekday_Name
application_date,,,,,,
2019-08-02,ASSAM,0.0,2,2019,8,Friday
2019-10-06,GUJARAT,0.0,6,2019,10,Sunday
2019-10-15,MADHYA PRADESH,0.0,15,2019,10,Tuesday
2019-08-01,JHARKHAND,0.0,1,2019,8,Thursday
2019-08-23,ORISSA,0.0,23,2019,8,Friday


In [0]:
df_test_h.reset_index(inplace=True)

In [1411]:
df_test_h['application_date'] = df_test_h.application_date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))
df_test_h.sample(5)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name
892,09/17/2019,MADHYA PRADESH,0.0,17,2019,9,Tuesday
1192,10/08/2019,TAMIL NADU,1.0,8,2019,10,Tuesday
1126,08/03/2019,TAMIL NADU,0.0,3,2019,8,Saturday
591,08/26/2019,KERALA,0.0,26,2019,8,Monday
309,08/23/2019,GUJARAT,0.0,23,2019,8,Friday


In [0]:
############## LABEL ENCODING ################################

In [1413]:
df_test_h.shape

(1488, 7)

In [0]:
label_encoder = preprocessing.LabelEncoder() 

In [0]:
#df_test_h['branch_id'] = label_encoder.fit_transform(df_test_h['branch_id'])
df_test_h['Holiday_Flag'] = label_encoder.fit_transform(df_test_h['Holiday_Flag'])
df_test_h['Weekday_Name'] = label_encoder.fit_transform(df_test_h['Weekday_Name'])
#df_test_h['zone'] = label_encoder.fit_transform(df_test_h['zone'])
df_test_h['state'] = label_encoder.fit_transform(df_test_h['state'])


In [1416]:
df_test_h.sample(5)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name
94,07/25/2019,1,0,25,2019,7,4
1365,09/25/2019,14,0,25,2019,9,6
290,08/04/2019,3,0,4,2019,8,3
324,09/07/2019,3,0,7,2019,9,2
477,08/05/2019,5,0,5,2019,8,1


In [0]:
## One HOT ENCODING of Testing Data

In [0]:
#df_test_h_dummy = pd.concat([df_test_h,pd.get_dummies(df_test_h['branch_id'], prefix='BR',dummy_na=False)],axis=1)
#df_test_h_dummy = pd.concat([df_test_h_dummy,pd.get_dummies(df_test_h_dummy['state'], prefix='ST',dummy_na=False)],axis=1)
#df_test_h_dummy = pd.concat([df_test_h_dummy,pd.get_dummies(df_test_h_dummy['zone'], prefix='ZN',dummy_na=False)],axis=1)
#df_test_h_dummy = pd.concat([df_test_h_dummy,pd.get_dummies(df_test_h_dummy['Holiday_name'], prefix='HD',dummy_na=False)],axis=1)
#df_test_h_dummy = pd.concat([df_test_h_dummy,pd.get_dummies(df_test_h_dummy['day'], prefix='Day',dummy_na=False)],axis=1)
#df_test_h_dummy = pd.concat([df_test_h_dummy,pd.get_dummies(df_test_h_dummy['Year'], prefix='YR',dummy_na=False)],axis=1)
#df_test_h_dummy = pd.concat([df_test_h_dummy,pd.get_dummies(df_test_h_dummy['Month'], prefix='MTH',dummy_na=False)],axis=1)
#df_test_h_dummy = pd.concat([df_test_h_dummy,pd.get_dummies(df_test_h_dummy['Weekday_Name'], prefix='WD',dummy_na=False)],axis=1)

In [1419]:
df_test_h.sample(5, random_state=0)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name
9,08/02/2019,0,0,2,2019,8,0
353,10/06/2019,3,0,6,2019,10,3
920,10/15/2019,8,0,15,2019,10,5
473,08/01/2019,5,0,1,2019,8,4
960,08/23/2019,10,0,23,2019,8,0


In [0]:
# df_train_nw_case_count_datewise['Weekday_Name'] = label_encoder.fit_transform(df_train_nw_case_count_datewise['Weekday_Name'])
# df_train_nw_case_count_datewise['Weekday_Name'].unique()

In [0]:
#df_test_h_dummy.drop(columns=['branch_id','state','zone','Holiday_name','day','Year','Month','Weekday_Name'],inplace=True)

In [0]:
# df_test_h_dummy.sample(5, random_state=0)

In [0]:
# df_train_h_dummy.sample(5, random_state=0)

In [0]:
df_test_h_dummy = df_test_h.copy(deep=True)

In [1425]:
print(df_train_h_dummy.shape, '|', df_test_h_dummy.shape )

(13504, 8) | (1488, 7)


In [0]:
#from google.colab import files

#df_train_h_dummy.iloc[:5,:].to_csv('/content/drive/My Drive/Hackathon/LTFS/25-JAN/Submission_Seg1/' + 'Train.csv')
# files.download('/content/drive/My Drive/Hackathon/LTFS/25-JAN/Submission_Seg1/' + 'Train.csv')

In [0]:
#from google.colab import files

#df_test_h_dummy.iloc[:5,:].to_csv('/content/drive/My Drive/Hackathon/LTFS/25-JAN/Submission_Seg1/' + 'Test.csv')
# files.download('/content/drive/My Drive/Hackathon/LTFS/25-JAN/Submission_Seg1/' + 'Test.csv')

In [0]:
#df_test_h_dummy.drop(columns='HD_Bakri Id / Eid ul-Adha',inplace=True)

In [0]:
#dummy_col = pd.DataFrame(columns=['HD_Buddha Purnima/Vesak','HD_Budhha Purnima','HD_Christmas','HD_Christmas Day','HD_Diwali (Deepavali)','HD_Diwali / Deepavali','HD_Dussehra','HD_Dussehra / Dasara','HD_Eid-ul-Fitar','HD_Good Friday','HD_Guru Nanaks Birthday','HD_Holi','HD_Id-ul-Zuha(Bakrid)','HD_Idul Fitr','HD_Janmashtarni (Vaishnav)','HD_Labor Day','HD_Maha Shivaratri','HD_Maha Shivaratri/Shivaratri','HD_Mahatma Gandhi Jayanti','HD_Mahavir Jayanti','HD_Makar Sankranti / Pongal','HD_Mathatma Gandhi Jayanti','HD_Milad un Nabi','HD_Milad-un-Nabi or Id-e- Milad (birthday of Prophet Mohammad)','HD_Muharram/Ashura','HD_New Years Day','HD_Raksha Bandhan','HD_Ram Navami','HD_Rama Navami','HD_Rath Yatra','HD_Republic Day','HD_Ugadi / Gudi Padwa','YR_2017','YR_2018','MTH_1','MTH_2','MTH_3','MTH_4','MTH_5','MTH_6','MTH_10','MTH_11','MTH_12'])

In [0]:
#df_test_h_dummy_col = pd.concat([df_test_h_dummy,dummy_col],axis=1)

In [0]:
#df_test_h_dummy_col.head()

In [0]:
#df_test_h_dummy_col.isnull().sum()

In [0]:
#df_test_h_dummy_col.replace(np.NaN,0,inplace=True)

In [1434]:
df_test_h_dummy.isnull().sum()

application_date    0
state               0
Holiday_Flag        0
day                 0
Year                0
Month               0
Weekday_Name        0
dtype: int64

In [0]:
#df_test_h_dummy_col.head(5)

In [0]:
df_test_h_dummy_col = df_test_h_dummy.copy(deep=True)

In [1437]:
df_train_h_dummy.head()

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name
0,04/01/2017,0,0,0,1,2017,4,2
1,04/02/2017,0,0,0,2,2017,4,3
2,04/03/2017,0,0,0,3,2017,4,1
3,04/04/2017,0,0,0,4,2017,4,5
4,04/05/2017,0,0,1,5,2017,4,6


In [1438]:
df_test_h_dummy_col.head()

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name
0,07/24/2019,0,0,24,2019,7,6
1,07/25/2019,0,0,25,2019,7,4
2,07/26/2019,0,0,26,2019,7,0
3,07/27/2019,0,0,27,2019,7,2
4,07/28/2019,0,0,28,2019,7,3


In [1439]:
print(df_train_h_dummy.shape," | ", df_test_h_dummy_col.shape)

(13504, 8)  |  (1488, 7)


In [0]:
####################################################################### Adding the LT&FS STOCKS ###################################################################

In [0]:
df_ltfs_stocks =  pd.read_csv('/content/drive/My Drive/Hackathon/LTFS/L&TFH_stocks.csv')

In [1442]:
df_ltfs_stocks.head(3)

,Date,Open Price,High Price,Low Price,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
0,31-Oct-19,94.6,98.45,94.2,95.65,96.083515,893176,3871,85819490,135190,15.14,4.25,1.05
1,30-Oct-19,95.0,95.70,93.0,93.60,94.551669,558398,2402,52797463,144533,25.88,2.70,-1.40
2,29-Oct-19,89.5,95.55,89.3,94.85,93.473841,1056242,4814,98730997,409376,38.76,6.25,5.35


In [0]:
df_ltfs_stocks['Date'] = df_ltfs_stocks['Date'].astype('datetime64[ns]')


In [0]:
df_ltfs_stocks['Date'] = df_ltfs_stocks.Date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [1445]:
df_ltfs_stocks.sample(4)

,Date,Open Price,High Price,Low Price,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
119,05/07/2019,125.00,125.45,122.00,122.45,123.906023,389405,2403,48249625,148674,38.18,3.45,-2.55
482,11/14/2017,180.80,186.65,179.10,184.60,183.307159,582284,3615,106736826,138487,23.78,7.55,3.80
561,07/20/2017,152.05,154.90,151.20,152.85,153.669246,670565,4254,103045218,196754,29.34,3.70,0.80
499,10/19/2017,207.90,207.90,203.75,205.25,206.180925,127589,1237,26306418,67900,53.22,4.15,-2.65


In [0]:
df_train_h_dummy_stock = pd.merge(df_train_h_dummy,df_ltfs_stocks,how='left',left_on='application_date',right_on='Date')

In [0]:
df_test_h_dummy_stock = pd.merge(df_test_h_dummy_col,df_ltfs_stocks,how='left',left_on='application_date',right_on='Date')

In [1448]:
print(df_train_h_dummy_stock.shape," | ", df_test_h_dummy_stock.shape)

(13504, 21)  |  (1488, 20)


In [1449]:
df_train_h_dummy_stock.sample(4)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Date,Open Price,High Price,Low Price,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
3408,05/03/2017,4,0,0,3,2017,5,6,05/03/2017,134.5,135.75,132.25,134.05,133.800125,683941.0,4192.0,91511391.0,187222.0,27.37,3.50,-0.45
5895,07/11/2019,7,1511,0,11,2019,7,4,07/11/2019,119.5,120.85,117.70,119.90,119.369205,494086.0,2723.0,58978653.0,80521.0,16.30,3.15,0.40
10638,08/24/2018,12,8038,0,24,2018,8,0,08/24/2018,184.9,186.15,179.20,180.65,181.618401,621891.0,4006.0,112946849.0,152822.0,24.57,6.95,-4.25
379,04/15/2018,0,110,0,15,2018,4,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1450]:
df_test_h_dummy_stock.sample(4)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name,Date,Open Price,High Price,Low Price,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
409,08/30/2019,4,0,30,2019,8,0,08/30/2019,94.15,94.7,91.55,94.35,93.158687,924340.0,2930.0,86110301.0,71682.0,7.75,3.15,0.2
867,08/23/2019,8,0,23,2019,8,0,08/23/2019,93.00,93.4,88.60,89.90,90.107863,1541446.0,7129.0,138896405.0,260097.0,16.87,4.80,-3.1
30,08/23/2019,0,0,23,2019,8,0,08/23/2019,93.00,93.4,88.60,89.90,90.107863,1541446.0,7129.0,138896405.0,260097.0,16.87,4.80,-3.1
145,09/14/2019,1,0,14,2019,9,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:


df_train_h_dummy_stock.drop(columns=['Open Price','High Price','Low Price','Date'],inplace=True)
df_test_h_dummy_stock.drop(columns=['Open Price','High Price','Low Price','Date'],inplace=True)

In [0]:
df_train_h_dummy_stock.fillna(method='ffill',inplace=True)
df_train_h_dummy_stock.fillna(method='bfill',inplace=True)
############################################################
df_test_h_dummy_stock.fillna(method='ffill',inplace=True)
df_test_h_dummy_stock.fillna(method='bfill',inplace=True)

In [1453]:
print(df_train_h_dummy_stock.isnull().sum().sum()," | " ,df_test_h_dummy_stock.isnull().sum().sum() )

0  |  0


In [1454]:
df_train_h_dummy_stock.sample(5)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
918,06/14/2017,1,0,0,14,2017,6,6,137.15,135.430130,925539.0,4246.0,125345867.0,365692.0,39.51,6.50,4.35
9381,07/07/2017,11,0,0,7,2017,7,0,148.40,148.310007,1446193.0,2110.0,214484894.0,97292.0,6.73,2.40,-0.75
2995,07/08/2018,3,204,0,8,2018,7,3,150.05,149.268216,380209.0,2004.0,56753119.0,82228.0,21.63,6.95,5.05
4192,06/26/2019,4,0,0,26,2019,6,6,115.55,114.715812,465235.0,2477.0,53369811.0,121417.0,26.10,4.05,2.05
1203,03/26/2018,1,1263,0,26,2018,3,1,155.80,154.367444,329917.0,1685.0,50928444.0,166406.0,50.44,5.10,1.95


In [1455]:
df_test_h_dummy_stock.sample(6)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
809,09/27/2019,9,0,27,2019,9,0,93.15,94.078951,774200.0,2828.0,72835924.0,152125.0,19.65,3.25,-2.85
456,10/16/2019,4,0,16,2019,10,6,83.05,83.284787,1012279.0,2858.0,84307441.0,126017.0,12.45,3.15,0.05
1167,09/13/2019,12,0,13,2019,9,0,93.35,92.484279,1602457.0,6687.0,148202080.0,184990.0,11.54,6.15,-3.60
11,08/04/2019,0,0,4,2019,8,3,95.25,94.943422,1074962.0,5056.0,102060571.0,136429.0,12.69,4.20,-0.25
353,10/06/2019,3,0,6,2019,10,3,80.60,81.632633,801575.0,3576.0,65434678.0,166396.0,20.76,3.80,-2.40
1277,09/30/2019,13,0,30,2019,9,1,84.75,86.916348,1833508.0,8669.0,159361820.0,505607.0,27.58,9.95,-8.45


In [1456]:
print(df_train_h_dummy_stock.shape,"| ",df_test_h_dummy_stock.shape)

(13504, 17) |  (1488, 16)


In [0]:
#################################################### Adding Sensex data to my training and Testing data ###################################################

In [0]:
df_sx =  pd.read_csv('/content/drive/My Drive/Hackathon/LTFS/Sensex.csv')

In [1459]:
df_sx.head(3)

,Date,Open,High,Low,Close
0,3-Apr-17,29737.73,29926.94,29705.72,29910.22
1,5-Apr-17,29996.03,30007.48,29817.69,29974.24
2,6-Apr-17,29946.89,29954.25,29817.59,29927.34


In [0]:
df_sx['Date'] = df_sx['Date'].astype('datetime64[ns]')


In [0]:
df_sx.drop(columns=['Open','High','Low'],inplace=True)

In [0]:
df_sx['Date'] = df_sx.Date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [1463]:
df_sx.head()

,Date,Close
0,04/03/2017,29910.22
1,04/05/2017,29974.24
2,04/06/2017,29927.34
3,04/07/2017,29706.61
4,04/10/2017,29575.74


In [1464]:
df_train_h_dummy_stock.head(4)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
0,04/01/2017,0,0,0,1,2017,4,2,123.8,123.822647,209260.0,1354.0,25911127.0,76967.0,36.78,1.55,0.6
1,04/02/2017,0,0,0,2,2017,4,3,123.8,123.822647,209260.0,1354.0,25911127.0,76967.0,36.78,1.55,0.6
2,04/03/2017,0,0,0,3,2017,4,1,123.8,123.822647,209260.0,1354.0,25911127.0,76967.0,36.78,1.55,0.6
3,04/04/2017,0,0,0,4,2017,4,5,123.8,123.822647,209260.0,1354.0,25911127.0,76967.0,36.78,1.55,0.6


In [1465]:
df_test_h_dummy_stock.tail(4)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open
1484,10/21/2019,15,0,21,2019,10,1,85.35,85.053695,1963540.0,4169.0,167006333.0,1215320.0,61.89,2.90,-1.35
1485,10/22/2019,15,0,22,2019,10,5,86.65,87.744178,1759496.0,7101.0,154385530.0,382961.0,21.77,6.85,2.45
1486,10/23/2019,15,0,23,2019,10,6,87.65,87.487553,582880.0,2699.0,50994745.0,80845.0,13.87,3.00,0.65
1487,10/24/2019,15,0,24,2019,10,4,88.40,88.900749,687377.0,2940.0,61108330.0,103611.0,15.07,2.60,0.30


In [0]:
df_train_h_dummy_stock_sx = pd.merge(df_train_h_dummy_stock,df_sx,how='left',left_on='application_date',right_on='Date')
df_test_h_dummy_stock_sx = pd.merge(df_test_h_dummy_stock,df_sx,how='left',left_on='application_date',right_on='Date')

In [1467]:
df_train_h_dummy_stock_sx.tail(4)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,Date,Close
13500,07/20/2019,15,1480,0,20,2019,7,2,110.5,113.376840,1041395.0,5601.0,118070074.0,249281.0,23.94,7.65,-6.9,NaN,NaN
13501,07/21/2019,15,1028,0,21,2019,7,3,110.5,113.376840,1041395.0,5601.0,118070074.0,249281.0,23.94,7.65,-6.9,NaN,NaN
13502,07/22/2019,15,1946,0,22,2019,7,1,108.1,108.083037,1640214.0,6255.0,177279311.0,228296.0,13.92,4.95,0.3,07/22/2019,38031.13
13503,07/23/2019,15,1984,0,23,2019,7,5,107.5,107.934062,680077.0,3086.0,73403473.0,115725.0,17.02,2.30,-1.0,07/23/2019,37982.74


In [1468]:
df_test_h_dummy_stock_sx.tail(4)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,Date,Close
1484,10/21/2019,15,0,21,2019,10,1,85.35,85.053695,1963540.0,4169.0,167006333.0,1215320.0,61.89,2.90,-1.35,NaN,NaN
1485,10/22/2019,15,0,22,2019,10,5,86.65,87.744178,1759496.0,7101.0,154385530.0,382961.0,21.77,6.85,2.45,10/22/2019,38963.84
1486,10/23/2019,15,0,23,2019,10,6,87.65,87.487553,582880.0,2699.0,50994745.0,80845.0,13.87,3.00,0.65,10/23/2019,39058.83
1487,10/24/2019,15,0,24,2019,10,4,88.40,88.900749,687377.0,2940.0,61108330.0,103611.0,15.07,2.60,0.30,10/24/2019,39020.39


In [0]:
df_train_h_dummy_stock_sx.drop(columns='Date',inplace=True)
df_test_h_dummy_stock_sx.drop(columns='Date',inplace=True)

In [1470]:
df_train_h_dummy_stock_sx.shape

(13504, 18)

In [1471]:
df_test_h_dummy_stock_sx.shape

(1488, 17)

In [0]:
df_train_h_dummy_stock_sx.fillna(method='ffill',inplace = True)
df_train_h_dummy_stock_sx.fillna(method='ffill',inplace = True)

df_test_h_dummy_stock_sx.fillna(method='bfill',inplace = True)
df_test_h_dummy_stock_sx.fillna(method='bfill',inplace = True)

df_train_h_dummy_stock_sx.fillna(method='ffill',inplace = True)
df_train_h_dummy_stock_sx.fillna(method='ffill',inplace = True)

In [1473]:
print(df_train_h_dummy_stock_sx.isnull().sum().sum(),"| ",df_test_h_dummy_stock_sx.isnull().sum().sum())


2 |  0


In [1474]:
df_train_h_dummy_stock_sx[(df_train_h_dummy_stock_sx['Close'].isnull())==True]

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,Close
0,04/01/2017,0,0,0,1,2017,4,2,123.8,123.822647,209260.0,1354.0,25911127.0,76967.0,36.78,1.55,0.6,NaN
1,04/02/2017,0,0,0,2,2017,4,3,123.8,123.822647,209260.0,1354.0,25911127.0,76967.0,36.78,1.55,0.6,NaN


In [0]:
df_train_h_dummy_stock_sx.loc[0,'Close']=df_train_h_dummy_stock_sx.loc[2,'Close']
df_train_h_dummy_stock_sx.loc[1,'Close']=df_train_h_dummy_stock_sx.loc[2,'Close']

In [1476]:
df_train_h_dummy_stock_sx[(df_train_h_dummy_stock_sx['Close'].isnull())==True]

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,Close


In [1477]:
print(df_train_h_dummy_stock_sx.isnull().sum().sum(),"| ",df_test_h_dummy_stock_sx.isnull().sum().sum())

0 |  0


In [1478]:
df_train_h_dummy_stock_sx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13504 entries, 0 to 13503
Data columns (total 18 columns):
application_date             13504 non-null object
state                        13504 non-null int64
case_count                   13504 non-null int64
Holiday_Flag                 13504 non-null int64
day                          13504 non-null int64
Year                         13504 non-null int64
Month                        13504 non-null int64
Weekday_Name                 13504 non-null int64
Close Price                  13504 non-null float64
WAP                          13504 non-null float64
No.of Shares                 13504 non-null float64
No. of Trades                13504 non-null float64
Total Turnover (Rs.)         13504 non-null float64
Deliverable Quantity         13504 non-null float64
% Deli. Qty to Traded Qty    13504 non-null float64
Spread High-Low              13504 non-null float64
Spread Close-Open            13504 non-null float64
Close                 

In [1479]:
print(df_train_h_dummy_stock_sx.shape,"| ",df_test_h_dummy_stock_sx.shape)

(13504, 18) |  (1488, 17)


In [0]:
df_train_h_dummy_stock_sx.rename(columns={'Close':'sx_Close'},inplace=True)
df_test_h_dummy_stock_sx.rename(columns={'Close':'sx_Close'},inplace=True)

In [1481]:
df_train_h_dummy_stock_sx.sample(4)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close
10361,11/20/2017,12,8280,0,20,2017,11,1,188.05,188.590464,222552.0,1332.0,41971185.0,71815.0,32.27,4.80,-2.95,33359.90
7371,12/11/2018,9,427,0,11,2018,12,5,137.70,136.022754,634564.0,3076.0,86315143.0,53942.0,8.50,7.45,5.70,35150.01
8954,08/28/2018,10,1251,0,28,2018,8,5,183.90,182.900038,415230.0,2578.0,75945583.0,111787.0,26.92,3.80,2.65,38896.63
11110,08/17/2017,13,0,0,17,2017,8,4,175.75,176.380377,449841.0,2909.0,79343125.0,104324.0,23.19,3.60,-1.30,31795.46


In [1482]:
df_test_h_dummy_stock_sx.sample(4)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close
487,08/15/2019,5,1,15,2019,8,4,103.80,102.935271,670931.0,3170.0,69062464.0,66238.0,9.87,3.45,1.90,37350.33
262,10/08/2019,2,1,8,2019,10,5,80.20,80.549128,1418823.0,5417.0,114284955.0,151920.0,10.71,2.60,-1.80,38177.95
1070,09/09/2019,11,0,9,2019,9,1,92.90,92.427793,715019.0,2794.0,66087628.0,112791.0,15.77,2.70,0.70,37145.45
1380,10/10/2019,14,0,10,2019,10,4,81.45,81.304772,547305.0,2370.0,44498508.0,88264.0,16.13,3.25,-1.95,37880.40


In [1483]:
print(df_train_h_dummy_stock_sx.shape,"| ",df_test_h_dummy_stock_sx.shape)

(13504, 18) |  (1488, 17)


In [0]:
################################################################### Adding the BANK RATES to our dataframe ##################################################################

In [0]:
bank_rates_file = "/content/drive/My Drive/Hackathon/LTFS/Bank_Rates.csv"

In [0]:
df_bk_rates = pd.read_csv(bank_rates_file)

In [1487]:

df_bk_rates.shape

(32, 87)

In [1488]:
df_bk_rates.head(5)

,Date,Allahabad Bank,Andhra Bank,Bank of Baroda,Bank of India,Bank of Maharashtra,Canara Bank,Central Bank of India,Corporation Bank,Dena Bank,IDBI Bank,Indian Bank,Indian Overseas Bank,Oriental Bank of Commerce,Punjab and Sind Bank,Punjab National Bank,State Bank of India,Syndicate Bank,UCO Bank,Union Bank of India,United Bank of India,Vijaya Bank,Axis Bank Ltd.,Bandhan Bank Ltd.,Catholic Syrian Bank Ltd.,City Union Bank Ltd.,Development Credit Bank Ltd.,Dhanalaxmi Bank Ltd.,Federal Bank Ltd.,HDFC Bank Ltd.,ICICI Bank Ltd.,IDFC Bank Ltd,Indusind Bank,Jammu and Kashmir Bank Ltd.,Karnataka Bank Ltd.,Karur Vysya Bank Ltd.,Kotak Mahindra Bank,Laxmi Vilas Bank Ltd.,Nainital Bank,RBL Bank,South Indian Bank Ltd.,Tamilnad Mercantile Bank Ltd.,Yes Bank Ltd.,AB Bank Ltd.,Abu Dhabi Commercial Bank Ltd.,American Express Banking Corporation,Australia and New zealand banking group Ltd.,Bank International Indonesia,Bank of Bahrain and Kuwait,Bank of America,Bank of Ceylon,Bank of Nova Scotia,Bank of Tokyo Mits UFJ Ltd.,Barclays Bank,BNP Paribas,Citi Bank,Credit Agricole Corporate & Investment Bank,Credit Suisse AG Bank,CTBC Bank Co.Ltd,Deutsche Bank,Development Bank of Singapore,Doha Bank QSC,Emirates NBD Bank (P.J.S.C),First Abu Dhabi Bank PJSC,Firstrand Bank Ltd,Hongkong & Shanghai Bkg. Corpn,Industrial and Commercial Bank of China,Industrial Bank of Korea,JP Morgan Chase Bank,KEB Hana Bank,Krung Thai Bank PCL,Mashreq Bank,Mizuho Corporate Bank,National Australia Bank,Qatar National Bank S.A.Q,Rabobank International,Sber Bank,Shinhan Bank,Societe Generale,Sonali Bank Ltd.,Standard Chartered Bank,State Bank of Mauritius,Sumitomo Mitsui Banking Corporation,The Royal Bank of Scotland,United Overseas Bank,Westpac Banking Corporation,Woori Bank
0,4/1/2017,8.6,8.65,8.35,8.5,8.75,8.45,8.5,8.75,8.60,8.80,8.5,8.65,8.60,8.75,8.45,8.0,8.75,8.60,8.5,8.80,8.65,8.25,9.95,9.9,9.50,9.98,9.90,8.95,8.15,8.2,8.6,9.1,8.60,8.9,9.50,8.80,9.75,8.6,9.75,9.1,9.50,8.80,7.15,8.65,7.38,7.90,9.25,8.40,7.80,10.16,8.53,7.05,7.90,8.45,7.95,7.91,7.30,7.68,9.30,8.1,8.20,NaN,8.5,8.40,7.83,9.00,8.2,8.35,8.5,8.25,7.7,7.45,7.6,NaN,7.95,9.16,8.2,7.90,7.23,8.95,8.90,7.20,8.45,8.50,9.0,7.9
1,5/1/2017,8.6,8.65,8.35,8.4,8.75,8.45,8.4,8.75,8.65,8.65,8.5,8.55,8.60,8.75,8.35,8.0,8.75,8.60,8.5,8.75,8.65,8.25,10.50,9.9,9.25,9.98,9.90,8.95,8.15,8.2,8.6,9.1,8.65,8.9,9.50,8.80,9.70,8.5,9.60,9.1,9.50,8.80,7.15,8.75,7.32,7.95,9.25,8.30,7.80,10.05,8.58,6.95,7.85,8.45,7.95,7.97,7.30,8.19,9.30,8.2,8.34,NaN,8.5,8.40,7.83,8.91,8.2,8.25,8.5,8.75,7.5,7.40,7.5,NaN,7.80,9.16,8.2,7.85,7.03,8.95,8.90,7.35,8.55,8.50,9.0,7.8
2,6/1/2017,8.5,8.65,8.35,8.4,8.75,8.40,8.4,8.75,8.60,8.65,8.5,8.55,8.50,8.75,8.35,8.0,8.75,8.60,8.4,8.75,8.65,8.25,10.50,9.9,9.25,9.72,9.90,8.90,8.15,8.2,8.6,9.1,8.90,8.9,9.35,8.90,9.50,8.5,9.65,9.1,9.45,8.90,7.15,8.60,7.31,7.35,9.25,8.30,7.95,9.94,8.76,7.10,7.85,8.45,8.20,7.88,7.33,8.08,9.30,8.2,8.64,NaN,7.4,8.57,7.82,9.03,8.2,8.60,8.5,8.50,7.7,7.80,7.5,NaN,7.95,9.16,8.0,7.80,7.60,8.85,8.90,7.35,8.70,8.19,8.5,7.8
3,7/1/2017,8.5,8.65,8.35,8.4,8.75,8.40,8.4,8.75,8.55,8.65,8.5,8.55,8.45,8.75,8.35,8.0,8.60,8.55,8.4,8.75,8.65,8.25,10.50,9.9,9.25,9.72,9.85,8.90,8.15,8.2,8.6,9.1,8.90,8.9,9.10,8.65,9.50,8.5,9.55,9.1,9.40,8.55,8.08,8.70,7.35,7.45,9.25,8.45,8.00,9.87,8.90,7.20,7.80,8.45,8.15,7.74,7.55,8.02,9.25,8.4,8.25,NaN,7.4,8.21,8.03,8.96,7.2,8.40,8.5,8.50,7.7,7.70,7.6,NaN,7.85,9.16,8.0,7.85,8.84,8.60,8.75,7.55,8.95,8.06,8.5,7.8
4,8/1/2017,8.5,8.65,8.35,8.4,8.75,8.40,8.4,8.75,8.55,8.65,8.5,8.55,8.45,8.75,8.35,8.0,8.50,8.55,8.4,8.75,8.65,8.25,10.45,9.8,9.25,9.72,9.85,8.90,8.15,8.2,8.6,9.0,8.90,8.9,9.10,8.65,9.45,8.4,9.55,9.0,9.40,8.85,8.08,8.75,6.79,7.35,9.25,8.35,7.95,9.80,8.25,7.10,7.50,8.30,8.15,7.48,7.44,8.03,9.25,8.2,8.05,8.05,7.3,8.24,8.10,8.86,8.0,8.10,8.5,8.25,7.8,7.40,7.4,7.35,7.65,9.16,8.0,7.70,8.84,8.60,8.75,7.20,8.70,8.08,8.5,7.8


In [0]:
df_bk_rates['Date'] = df_bk_rates['Date'].astype('datetime64[ns]')

In [0]:
df_bk_rates['Date'] = df_bk_rates.Date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [1491]:
df_bk_rates.head(4)

,Date,Allahabad Bank,Andhra Bank,Bank of Baroda,Bank of India,Bank of Maharashtra,Canara Bank,Central Bank of India,Corporation Bank,Dena Bank,IDBI Bank,Indian Bank,Indian Overseas Bank,Oriental Bank of Commerce,Punjab and Sind Bank,Punjab National Bank,State Bank of India,Syndicate Bank,UCO Bank,Union Bank of India,United Bank of India,Vijaya Bank,Axis Bank Ltd.,Bandhan Bank Ltd.,Catholic Syrian Bank Ltd.,City Union Bank Ltd.,Development Credit Bank Ltd.,Dhanalaxmi Bank Ltd.,Federal Bank Ltd.,HDFC Bank Ltd.,ICICI Bank Ltd.,IDFC Bank Ltd,Indusind Bank,Jammu and Kashmir Bank Ltd.,Karnataka Bank Ltd.,Karur Vysya Bank Ltd.,Kotak Mahindra Bank,Laxmi Vilas Bank Ltd.,Nainital Bank,RBL Bank,South Indian Bank Ltd.,Tamilnad Mercantile Bank Ltd.,Yes Bank Ltd.,AB Bank Ltd.,Abu Dhabi Commercial Bank Ltd.,American Express Banking Corporation,Australia and New zealand banking group Ltd.,Bank International Indonesia,Bank of Bahrain and Kuwait,Bank of America,Bank of Ceylon,Bank of Nova Scotia,Bank of Tokyo Mits UFJ Ltd.,Barclays Bank,BNP Paribas,Citi Bank,Credit Agricole Corporate & Investment Bank,Credit Suisse AG Bank,CTBC Bank Co.Ltd,Deutsche Bank,Development Bank of Singapore,Doha Bank QSC,Emirates NBD Bank (P.J.S.C),First Abu Dhabi Bank PJSC,Firstrand Bank Ltd,Hongkong & Shanghai Bkg. Corpn,Industrial and Commercial Bank of China,Industrial Bank of Korea,JP Morgan Chase Bank,KEB Hana Bank,Krung Thai Bank PCL,Mashreq Bank,Mizuho Corporate Bank,National Australia Bank,Qatar National Bank S.A.Q,Rabobank International,Sber Bank,Shinhan Bank,Societe Generale,Sonali Bank Ltd.,Standard Chartered Bank,State Bank of Mauritius,Sumitomo Mitsui Banking Corporation,The Royal Bank of Scotland,United Overseas Bank,Westpac Banking Corporation,Woori Bank
0,04/01/2017,8.6,8.65,8.35,8.5,8.75,8.45,8.5,8.75,8.60,8.80,8.5,8.65,8.60,8.75,8.45,8.0,8.75,8.60,8.5,8.80,8.65,8.25,9.95,9.9,9.50,9.98,9.90,8.95,8.15,8.2,8.6,9.1,8.60,8.9,9.50,8.80,9.75,8.6,9.75,9.1,9.50,8.80,7.15,8.65,7.38,7.90,9.25,8.40,7.80,10.16,8.53,7.05,7.90,8.45,7.95,7.91,7.30,7.68,9.30,8.1,8.20,NaN,8.5,8.40,7.83,9.00,8.2,8.35,8.5,8.25,7.7,7.45,7.6,NaN,7.95,9.16,8.2,7.90,7.23,8.95,8.90,7.20,8.45,8.50,9.0,7.9
1,05/01/2017,8.6,8.65,8.35,8.4,8.75,8.45,8.4,8.75,8.65,8.65,8.5,8.55,8.60,8.75,8.35,8.0,8.75,8.60,8.5,8.75,8.65,8.25,10.50,9.9,9.25,9.98,9.90,8.95,8.15,8.2,8.6,9.1,8.65,8.9,9.50,8.80,9.70,8.5,9.60,9.1,9.50,8.80,7.15,8.75,7.32,7.95,9.25,8.30,7.80,10.05,8.58,6.95,7.85,8.45,7.95,7.97,7.30,8.19,9.30,8.2,8.34,NaN,8.5,8.40,7.83,8.91,8.2,8.25,8.5,8.75,7.5,7.40,7.5,NaN,7.80,9.16,8.2,7.85,7.03,8.95,8.90,7.35,8.55,8.50,9.0,7.8
2,06/01/2017,8.5,8.65,8.35,8.4,8.75,8.40,8.4,8.75,8.60,8.65,8.5,8.55,8.50,8.75,8.35,8.0,8.75,8.60,8.4,8.75,8.65,8.25,10.50,9.9,9.25,9.72,9.90,8.90,8.15,8.2,8.6,9.1,8.90,8.9,9.35,8.90,9.50,8.5,9.65,9.1,9.45,8.90,7.15,8.60,7.31,7.35,9.25,8.30,7.95,9.94,8.76,7.10,7.85,8.45,8.20,7.88,7.33,8.08,9.30,8.2,8.64,NaN,7.4,8.57,7.82,9.03,8.2,8.60,8.5,8.50,7.7,7.80,7.5,NaN,7.95,9.16,8.0,7.80,7.60,8.85,8.90,7.35,8.70,8.19,8.5,7.8
3,07/01/2017,8.5,8.65,8.35,8.4,8.75,8.40,8.4,8.75,8.55,8.65,8.5,8.55,8.45,8.75,8.35,8.0,8.60,8.55,8.4,8.75,8.65,8.25,10.50,9.9,9.25,9.72,9.85,8.90,8.15,8.2,8.6,9.1,8.90,8.9,9.10,8.65,9.50,8.5,9.55,9.1,9.40,8.55,8.08,8.70,7.35,7.45,9.25,8.45,8.00,9.87,8.90,7.20,7.80,8.45,8.15,7.74,7.55,8.02,9.25,8.4,8.25,NaN,7.4,8.21,8.03,8.96,7.2,8.40,8.5,8.50,7.7,7.70,7.6,NaN,7.85,9.16,8.0,7.85,8.84,8.60,8.75,7.55,8.95,8.06,8.5,7.8


In [1492]:
df_train_h_dummy_stock_sx.head(4)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close
0,04/01/2017,0,0,0,1,2017,4,2,123.8,123.822647,209260.0,1354.0,25911127.0,76967.0,36.78,1.55,0.6,29910.22
1,04/02/2017,0,0,0,2,2017,4,3,123.8,123.822647,209260.0,1354.0,25911127.0,76967.0,36.78,1.55,0.6,29910.22
2,04/03/2017,0,0,0,3,2017,4,1,123.8,123.822647,209260.0,1354.0,25911127.0,76967.0,36.78,1.55,0.6,29910.22
3,04/04/2017,0,0,0,4,2017,4,5,123.8,123.822647,209260.0,1354.0,25911127.0,76967.0,36.78,1.55,0.6,29910.22


In [1493]:
df_test_h_dummy_stock_sx.tail(4)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close
1484,10/21/2019,15,0,21,2019,10,1,85.35,85.053695,1963540.0,4169.0,167006333.0,1215320.0,61.89,2.90,-1.35,38963.84
1485,10/22/2019,15,0,22,2019,10,5,86.65,87.744178,1759496.0,7101.0,154385530.0,382961.0,21.77,6.85,2.45,38963.84
1486,10/23/2019,15,0,23,2019,10,6,87.65,87.487553,582880.0,2699.0,50994745.0,80845.0,13.87,3.00,0.65,39058.83
1487,10/24/2019,15,0,24,2019,10,4,88.40,88.900749,687377.0,2940.0,61108330.0,103611.0,15.07,2.60,0.30,39020.39


In [0]:
df_train_h_dummy_stock_sx_rate = pd.merge(df_train_h_dummy_stock_sx,df_bk_rates,how='left',left_on='application_date',right_on='Date')
df_test_h_dummy_stock_sx_rate = pd.merge(df_test_h_dummy_stock_sx,df_bk_rates,how='left',left_on='application_date',right_on='Date')

In [1495]:
print(df_train_h_dummy_stock_sx_rate.shape ,' | ',df_test_h_dummy_stock_sx_rate.shape)

(13504, 105)  |  (1488, 104)


In [1496]:
df_train_h_dummy_stock_sx_rate.sample(4)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close,Date,Allahabad Bank,Andhra Bank,Bank of Baroda,Bank of India,Bank of Maharashtra,Canara Bank,Central Bank of India,Corporation Bank,Dena Bank,IDBI Bank,Indian Bank,Indian Overseas Bank,Oriental Bank of Commerce,Punjab and Sind Bank,Punjab National Bank,State Bank of India,Syndicate Bank,UCO Bank,Union Bank of India,United Bank of India,Vijaya Bank,Axis Bank Ltd.,Bandhan Bank Ltd.,Catholic Syrian Bank Ltd.,City Union Bank Ltd.,Development Credit Bank Ltd.,Dhanalaxmi Bank Ltd.,Federal Bank Ltd.,HDFC Bank Ltd.,ICICI Bank Ltd.,IDFC Bank Ltd,...,Laxmi Vilas Bank Ltd.,Nainital Bank,RBL Bank,South Indian Bank Ltd.,Tamilnad Mercantile Bank Ltd.,Yes Bank Ltd.,AB Bank Ltd.,Abu Dhabi Commercial Bank Ltd.,American Express Banking Corporation,Australia and New zealand banking group Ltd.,Bank International Indonesia,Bank of Bahrain and Kuwait,Bank of America,Bank of Ceylon,Bank of Nova Scotia,Bank of Tokyo Mits UFJ Ltd.,Barclays Bank,BNP Paribas,Citi Bank,Credit Agricole Corporate & Investment Bank,Credit Suisse AG Bank,CTBC Bank Co.Ltd,Deutsche Bank,Development Bank of Singapore,Doha Bank QSC,Emirates NBD Bank (P.J.S.C),First Abu Dhabi Bank PJSC,Firstrand Bank Ltd,Hongkong & Shanghai Bkg. Corpn,Industrial and Commercial Bank of China,Industrial Bank of Korea,JP Morgan Chase Bank,KEB Hana Bank,Krung Thai Bank PCL,Mashreq Bank,Mizuho Corporate Bank,National Australia Bank,Qatar National Bank S.A.Q,Rabobank International,Sber Bank,Shinhan Bank,Societe Generale,Sonali Bank Ltd.,Standard Chartered Bank,State Bank of Mauritius,Sumitomo Mitsui Banking Corporation,The Royal Bank of Scotland,United Overseas Bank,Westpac Banking Corporation,Woori Bank
5253,10/07/2017,7,377,0,7,2017,10,2,197.35,197.758062,695607.0,3683.0,137561892.0,184498.0,26.52,6.45,3.45,31814.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6300,04/28/2018,6,728,0,28,2018,4,2,168.75,168.520194,310565.0,1846.0,52336474.0,87152.0,28.06,4.40,3.50,34969.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7476,03/26/2019,9,478,0,26,2019,3,5,147.60,147.038970,410958.0,2003.0,60426841.0,81153.0,19.75,3.95,2.95,38233.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1312,07/13/2018,1,2977,0,13,2018,7,0,147.60,148.927099,406373.0,2221.0,60519952.0,122165.0,30.06,9.70,-8.65,36541.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df_train_h_dummy_stock_sx_rate.drop(columns='Date',inplace=True)
df_test_h_dummy_stock_sx_rate.drop(columns='Date',inplace=True)

In [1498]:
df_test_h_dummy_stock_sx_rate.sample(5)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close,Allahabad Bank,Andhra Bank,Bank of Baroda,Bank of India,Bank of Maharashtra,Canara Bank,Central Bank of India,Corporation Bank,Dena Bank,IDBI Bank,Indian Bank,Indian Overseas Bank,Oriental Bank of Commerce,Punjab and Sind Bank,Punjab National Bank,State Bank of India,Syndicate Bank,UCO Bank,Union Bank of India,United Bank of India,Vijaya Bank,Axis Bank Ltd.,Bandhan Bank Ltd.,Catholic Syrian Bank Ltd.,City Union Bank Ltd.,Development Credit Bank Ltd.,Dhanalaxmi Bank Ltd.,Federal Bank Ltd.,HDFC Bank Ltd.,ICICI Bank Ltd.,IDFC Bank Ltd,Indusind Bank,Jammu and Kashmir Bank Ltd.,...,Laxmi Vilas Bank Ltd.,Nainital Bank,RBL Bank,South Indian Bank Ltd.,Tamilnad Mercantile Bank Ltd.,Yes Bank Ltd.,AB Bank Ltd.,Abu Dhabi Commercial Bank Ltd.,American Express Banking Corporation,Australia and New zealand banking group Ltd.,Bank International Indonesia,Bank of Bahrain and Kuwait,Bank of America,Bank of Ceylon,Bank of Nova Scotia,Bank of Tokyo Mits UFJ Ltd.,Barclays Bank,BNP Paribas,Citi Bank,Credit Agricole Corporate & Investment Bank,Credit Suisse AG Bank,CTBC Bank Co.Ltd,Deutsche Bank,Development Bank of Singapore,Doha Bank QSC,Emirates NBD Bank (P.J.S.C),First Abu Dhabi Bank PJSC,Firstrand Bank Ltd,Hongkong & Shanghai Bkg. Corpn,Industrial and Commercial Bank of China,Industrial Bank of Korea,JP Morgan Chase Bank,KEB Hana Bank,Krung Thai Bank PCL,Mashreq Bank,Mizuho Corporate Bank,National Australia Bank,Qatar National Bank S.A.Q,Rabobank International,Sber Bank,Shinhan Bank,Societe Generale,Sonali Bank Ltd.,Standard Chartered Bank,State Bank of Mauritius,Sumitomo Mitsui Banking Corporation,The Royal Bank of Scotland,United Overseas Bank,Westpac Banking Corporation,Woori Bank
533,09/30/2019,5,0,30,2019,9,1,84.75,86.916348,1833508.0,8669.0,159361820.0,505607.0,27.58,9.95,-8.45,38667.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
703,09/14/2019,6,0,14,2019,9,2,93.35,92.484279,1602457.0,6687.0,148202080.0,184990.0,11.54,6.15,-3.60,37123.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,10/24/2019,1,0,24,2019,10,4,88.40,88.900749,687377.0,2940.0,61108330.0,103611.0,15.07,2.60,0.30,39020.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
849,08/05/2019,8,0,5,2019,8,1,94.80,94.628355,747974.0,3164.0,70779549.0,77342.0,10.34,4.95,-0.10,36699.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1340,08/31/2019,14,0,31,2019,8,2,94.35,93.158687,924340.0,2930.0,86110301.0,71682.0,7.75,3.15,0.20,36562.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [0]:
df_train_h_dummy_stock_sx_rate.fillna(method='bfill',inplace=True)
df_test_h_dummy_stock_sx_rate.fillna(method='bfill',inplace=True)

In [1500]:
print(df_train_h_dummy_stock_sx_rate.isnull().sum().sum() , ' | ', df_test_h_dummy_stock_sx_rate.isnull().sum().sum())

2348  |  6403


In [0]:
df_train_h_dummy_stock_sx_rate.fillna(method='ffill',inplace=True)
df_test_h_dummy_stock_sx_rate.fillna(method='ffill',inplace=True)

In [1502]:
print(df_train_h_dummy_stock_sx_rate.isnull().sum().sum() , ' | ', df_test_h_dummy_stock_sx_rate.isnull().sum().sum())

0  |  4464


In [1503]:
df_test_h_dummy_stock_sx_rate.iloc[:,19:].info() #Dena Bank,IDBI Bank,Vijaya Bank

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1488 entries, 0 to 1487
Data columns (total 84 columns):
Bank of Baroda                                  1488 non-null float64
Bank of India                                   1488 non-null float64
Bank of Maharashtra                             1488 non-null float64
Canara Bank                                     1488 non-null float64
Central Bank of India                           1488 non-null float64
Corporation Bank                                1488 non-null float64
Dena Bank                                       0 non-null float64
IDBI Bank                                       0 non-null float64
Indian Bank                                     1488 non-null float64
Indian Overseas Bank                            1488 non-null float64
Oriental Bank of Commerce                       1488 non-null float64
Punjab and Sind Bank                            1488 non-null float64
Punjab National Bank                            1488 non-nu

In [1504]:
df_test_h_dummy_stock_sx_rate.loc[:,['Dena Bank','IDBI Bank','Vijaya Bank']].head()

,Dena Bank,IDBI Bank,Vijaya Bank
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [0]:
df_train_h_dummy_stock_sx_rate.drop(columns=['Dena Bank','IDBI Bank','Vijaya Bank'],inplace=True)
df_test_h_dummy_stock_sx_rate.drop(columns=['Dena Bank','IDBI Bank','Vijaya Bank'],inplace=True)

In [1506]:
print(df_train_h_dummy_stock_sx_rate.isnull().sum().sum() , ' | ', df_test_h_dummy_stock_sx_rate.isnull().sum().sum())

0  |  0


In [0]:
#df_train_h_dummy_stock_sx_rate = pd.merge(df_train_h_dummy_stock_sx,df_bk_rates,how='left',left_on=['Month','Year'],right_on=['month','Year'])

In [1508]:
print(df_train_h_dummy_stock_sx_rate.shape , ' | ', df_test_h_dummy_stock_sx_rate.shape)

(13504, 101)  |  (1488, 100)


In [0]:
####################################################################### Adding the Gold Prices ###################################################################

In [0]:
df_gold =  pd.read_csv('/content/drive/My Drive/Hackathon/LTFS/Gold Prices.csv')

In [1511]:
df_gold.head(3)

,Gold_date,Gold_price
0,4/3/2017,81121.1
1,4/4/2017,81797.5
2,4/5/2017,80871.1


In [0]:
df_gold['Gold_date'] = df_gold['Gold_date'].astype('datetime64[ns]')


In [0]:
df_gold['Gold_date'] = df_gold.Gold_date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [1514]:

df_gold.sample(4)

,Gold_date,Gold_price
166,11/21/2017,83303.8
359,08/17/2018,82837.1
57,06/21/2017,80152.4
445,12/17/2018,88877.3


In [0]:
df_train_h_dummy_stock_sx_rate_gold = pd.merge(df_train_h_dummy_stock_sx_rate,df_gold,how='left',left_on='application_date',right_on='Gold_date')

In [0]:
df_test_h_dummy_stock_sx_rate_gold = pd.merge(df_test_h_dummy_stock_sx_rate,df_gold,how='left',left_on='application_date',right_on='Gold_date')

In [1517]:
print(df_train_h_dummy_stock_sx_rate_gold.shape," | ", df_test_h_dummy_stock_sx_rate_gold.shape)

(13504, 103)  |  (1488, 102)


In [1518]:

print(df_train_h_dummy_stock_sx_rate_gold.isnull().sum().sum() ," | ", df_test_h_dummy_stock_sx_rate_gold.isnull().sum().sum() )

7744  |  832


In [0]:
df_train_h_dummy_stock_sx_rate_gold.fillna(method='bfill',inplace=True)
df_test_h_dummy_stock_sx_rate_gold.fillna(method='bfill',inplace=True)

In [1520]:
print(df_train_h_dummy_stock_sx_rate_gold.isnull().sum().sum() , ' | ', df_test_h_dummy_stock_sx_rate_gold.isnull().sum().sum())

0  |  0


In [0]:
df_train_h_dummy_stock_sx_rate_gold.fillna(method='ffill',inplace=True)
df_test_h_dummy_stock_sx_rate_gold.fillna(method='ffill',inplace=True)

In [1522]:
print(df_train_h_dummy_stock_sx_rate_gold.isnull().sum().sum() , ' | ', df_test_h_dummy_stock_sx_rate_gold.isnull().sum().sum())

0  |  0


In [0]:
#df_test_h_dummy_stock_sx_rate.replace(np.NaN,np.mean,inplace=True)

In [1524]:
df_train_h_dummy_stock_sx_rate_gold.sample(4)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close,Allahabad Bank,Andhra Bank,Bank of Baroda,Bank of India,Bank of Maharashtra,Canara Bank,Central Bank of India,Corporation Bank,Indian Bank,Indian Overseas Bank,Oriental Bank of Commerce,Punjab and Sind Bank,Punjab National Bank,State Bank of India,Syndicate Bank,UCO Bank,Union Bank of India,United Bank of India,Axis Bank Ltd.,Bandhan Bank Ltd.,Catholic Syrian Bank Ltd.,City Union Bank Ltd.,Development Credit Bank Ltd.,Dhanalaxmi Bank Ltd.,Federal Bank Ltd.,HDFC Bank Ltd.,ICICI Bank Ltd.,IDFC Bank Ltd,Indusind Bank,Jammu and Kashmir Bank Ltd.,Karnataka Bank Ltd.,Karur Vysya Bank Ltd.,...,RBL Bank,South Indian Bank Ltd.,Tamilnad Mercantile Bank Ltd.,Yes Bank Ltd.,AB Bank Ltd.,Abu Dhabi Commercial Bank Ltd.,American Express Banking Corporation,Australia and New zealand banking group Ltd.,Bank International Indonesia,Bank of Bahrain and Kuwait,Bank of America,Bank of Ceylon,Bank of Nova Scotia,Bank of Tokyo Mits UFJ Ltd.,Barclays Bank,BNP Paribas,Citi Bank,Credit Agricole Corporate & Investment Bank,Credit Suisse AG Bank,CTBC Bank Co.Ltd,Deutsche Bank,Development Bank of Singapore,Doha Bank QSC,Emirates NBD Bank (P.J.S.C),First Abu Dhabi Bank PJSC,Firstrand Bank Ltd,Hongkong & Shanghai Bkg. Corpn,Industrial and Commercial Bank of China,Industrial Bank of Korea,JP Morgan Chase Bank,KEB Hana Bank,Krung Thai Bank PCL,Mashreq Bank,Mizuho Corporate Bank,National Australia Bank,Qatar National Bank S.A.Q,Rabobank International,Sber Bank,Shinhan Bank,Societe Generale,Sonali Bank Ltd.,Standard Chartered Bank,State Bank of Mauritius,Sumitomo Mitsui Banking Corporation,The Royal Bank of Scotland,United Overseas Bank,Westpac Banking Corporation,Woori Bank,Gold_date,Gold_price
10283,09/03/2017,12,1217,0,3,2017,9,3,198.70,200.110072,1136943.0,6591.0,227513745.0,266898.0,23.48,4.80,-2.30,31892.23,8.45,8.40,8.30,8.30,8.65,8.30,8.30,8.65,8.35,8.4,8.40,8.45,8.15,8.00,8.50,8.45,8.20,8.75,8.25,10.25,9.8,9.25,9.72,9.8,8.9,8.15,8.20,8.60,8.85,8.80,8.85,9.00,...,9.25,9.00,9.15,8.80,7.26,8.20,7.01,7.55,9.00,8.40,7.65,9.97,7.19,7.25,7.5,8.15,8.05,7.42,7.16,8.15,9.30,8.20,8.20,8.05,7.0,8.20,8.06,8.09,6.7,7.90,8.25,8.25,7.60,7.30,7.2,7.40,7.50,8.87,8.00,7.75,7.38,8.55,8.75,7.35,8.65,7.99,8.50,7.5,09/04/2017,85400.0
6944,10/10/2017,9,358,0,10,2017,10,5,202.85,200.917274,774928.0,3910.0,155696421.0,154073.0,19.88,7.55,6.75,31924.41,8.30,8.40,8.30,8.30,8.65,8.30,8.30,8.65,8.35,8.4,8.40,8.40,8.15,7.95,8.45,8.45,8.20,8.60,8.25,10.20,9.8,9.25,9.72,9.8,8.9,8.10,8.20,8.50,8.85,8.80,8.75,9.00,...,9.20,9.00,9.15,8.80,7.26,8.20,7.32,7.50,9.00,8.40,7.75,9.95,7.09,7.05,7.5,8.00,8.10,7.32,7.07,8.38,9.25,8.30,8.35,8.05,7.0,8.27,8.05,8.01,6.5,8.35,8.25,8.25,7.55,7.30,7.2,7.45,7.55,8.87,8.00,7.75,7.04,8.55,8.75,7.20,8.60,7.96,8.50,7.5,10/10/2017,84270.3
5563,08/13/2018,7,3175,0,13,2018,8,1,174.55,175.040700,344106.0,2674.0,60232555.0,95512.0,27.76,4.10,-1.95,37644.90,8.50,8.70,8.55,8.60,8.75,8.65,8.60,8.95,8.60,8.8,8.65,8.80,8.45,8.45,8.80,8.65,8.55,8.80,8.70,10.50,9.9,9.30,10.40,9.7,9.2,8.40,8.55,9.10,9.65,8.85,8.95,9.55,...,9.75,9.35,9.10,9.65,6.82,9.20,5.82,8.40,9.40,8.70,8.20,9.83,8.67,7.20,8.5,9.15,8.45,8.78,8.25,8.62,10.35,8.90,8.60,8.80,6.7,8.78,8.65,8.93,6.6,8.75,8.30,8.25,8.95,8.75,8.5,7.95,8.55,8.52,8.35,8.65,6.15,9.15,9.20,8.55,6.65,9.46,8.40,8.3,08/13/2018,83838.4
7398,01/07/2019,9,141,0,7,2019,1,1,145.80,146.728007,460233.0,2573.0,67529071.0,109551.0,23.80,3.60,-0.20,35850.16,8.75,8.75,8.75,8.75,8.75,8.70,8.65,8.95,8.75,8.8,8.75,8.85,8.55,8.55,8.75,8.70,8.70,8.85,8.90,10.45,9.9,9.30,10.84,9.9,9.2,8.75,8.80,9.25,9.90,9.00,9.15,9.65,...,10.25,9.45,9.20,9.80,6.70,9.65,7.22,8.25,9.85,8.65,7.80,9.87,8.85,7.90,8.9,9.50,8.80,8.17,8.30,7.70,10.30,8.85,9.15,8.70,7.3,9.15,9.15,9.05,5.9,9.00,8.45,8.25,8.90,8.30,9.3,8.10,8.50,8.52,8.60,8.45,7.02,9

In [1525]:
df_test_h_dummy_stock_sx_rate_gold.sample(4)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close,Allahabad Bank,Andhra Bank,Bank of Baroda,Bank of India,Bank of Maharashtra,Canara Bank,Central Bank of India,Corporation Bank,Indian Bank,Indian Overseas Bank,Oriental Bank of Commerce,Punjab and Sind Bank,Punjab National Bank,State Bank of India,Syndicate Bank,UCO Bank,Union Bank of India,United Bank of India,Axis Bank Ltd.,Bandhan Bank Ltd.,Catholic Syrian Bank Ltd.,City Union Bank Ltd.,Development Credit Bank Ltd.,Dhanalaxmi Bank Ltd.,Federal Bank Ltd.,HDFC Bank Ltd.,ICICI Bank Ltd.,IDFC Bank Ltd,Indusind Bank,Jammu and Kashmir Bank Ltd.,Karnataka Bank Ltd.,Karur Vysya Bank Ltd.,Kotak Mahindra Bank,...,RBL Bank,South Indian Bank Ltd.,Tamilnad Mercantile Bank Ltd.,Yes Bank Ltd.,AB Bank Ltd.,Abu Dhabi Commercial Bank Ltd.,American Express Banking Corporation,Australia and New zealand banking group Ltd.,Bank International Indonesia,Bank of Bahrain and Kuwait,Bank of America,Bank of Ceylon,Bank of Nova Scotia,Bank of Tokyo Mits UFJ Ltd.,Barclays Bank,BNP Paribas,Citi Bank,Credit Agricole Corporate & Investment Bank,Credit Suisse AG Bank,CTBC Bank Co.Ltd,Deutsche Bank,Development Bank of Singapore,Doha Bank QSC,Emirates NBD Bank (P.J.S.C),First Abu Dhabi Bank PJSC,Firstrand Bank Ltd,Hongkong & Shanghai Bkg. Corpn,Industrial and Commercial Bank of China,Industrial Bank of Korea,JP Morgan Chase Bank,KEB Hana Bank,Krung Thai Bank PCL,Mashreq Bank,Mizuho Corporate Bank,National Australia Bank,Qatar National Bank S.A.Q,Rabobank International,Sber Bank,Shinhan Bank,Societe Generale,Sonali Bank Ltd.,Standard Chartered Bank,State Bank of Mauritius,Sumitomo Mitsui Banking Corporation,The Royal Bank of Scotland,United Overseas Bank,Westpac Banking Corporation,Woori Bank,Gold_date,Gold_price
310,08/24/2019,3,1,24,2019,8,2,89.90,90.107863,1541446.0,7129.0,138896405.0,260097.0,16.87,4.80,-3.10,37494.12,8.4,8.4,8.4,8.35,8.5,8.4,8.25,8.65,8.45,8.5,8.4,8.5,8.3,8.15,8.35,8.5,8.35,8.45,8.45,10.11,9.9,9.0,10.65,9.9,8.98,8.45,8.55,9.3,9.55,8.7,9.25,9.45,8.75,...,9.95,9.25,9.1,9.7,7.85,9.3,6.76,7.45,9.4,8.65,7.95,9.8,7.97,6.95,8.3,8.3,8.45,7.2,8.0,7.04,9.55,8.3,8.7,8.0,7.1,8.68,8.5,8.29,7.0,7.8,8.45,8.0,7.04,7.8,8.3,8.7,8.0,7.1,8.68,8.5,8.29,9.35,7.8,7.0,7.15,8.28,8.75,8.5,08/26/2019,108299.9
129,08/29/2019,1,0,29,2019,8,4,93.05,93.618216,1040798.0,3463.0,97437652.0,253421.0,24.35,4.00,-2.45,37068.93,8.4,8.4,8.4,8.35,8.5,8.4,8.25,8.65,8.45,8.5,8.4,8.5,8.3,8.15,8.35,8.5,8.35,8.45,8.45,10.11,9.9,9.0,10.65,9.9,8.98,8.45,8.55,9.3,9.55,8.7,9.25,9.45,8.75,...,9.95,9.25,9.1,9.7,7.85,9.3,6.76,7.45,9.4,8.65,7.95,9.8,7.97,6.95,8.3,8.3,8.45,7.2,8.0,7.04,9.55,8.3,8.7,8.0,7.1,8.68,8.5,8.29,7.0,7.8,8.45,8.0,7.04,7.8,8.3,8.7,8.0,7.1,8.68,8.5,8.29,9.35,7.8,7.0,7.15,8.28,8.75,8.5,08/29/2019,110544.0
1234,08/18/2019,13,0,18,2019,8,3,105.00,104.220677,590556.0,2471.0,61548146.0,66144.0,11.20,3.55,1.20,37402.49,8.4,8.4,8.4,8.35,8.5,8.4,8.25,8.65,8.45,8.5,8.4,8.5,8.3,8.15,8.35,8.5,8.35,8.45,8.45,10.11,9.9,9.0,10.65,9.9,8.98,8.45,8.55,9.3,9.55,8.7,9.25,9.45,8.75,...,9.95,9.25,9.1,9.7,7.85,9.3,6.76,7.45,9.4,8.65,7.95,9.8,7.97,6.95,8.3,8.3,8.45,7.2,8.0,7.04,9.55,8.3,8.7,8.0,7.1,8.68,8.5,8.29,7.0,7.8,8.45,8.0,7.04,7.8,8.3,8.7,8.0,7.1,8.68,8.5,8.29,9.35,7.8,7.0,7.15,8.28,8.75,8.5,08/19/2019,106890.0
295,08/09/2019,3,0,9,2019,8,0,104.00,104.214696,1417914.0,7474.0,147767476.0,155559.0,10.97,7.10,3.00,37581.91,8.4,8.4,8.4,8.35,8.5,8.4,8.25,8.65,8.45,8.5,8.4,8.5,8.3,8.15,8.35,8.5,8.35,8.45,8.45,10.11,9.9,9.0,10.65,9.9,8.98,8.45,8.55,9.3,9.55,8.7,9.25,9.45,8.75,...,9.95,9.25,9.1,9.7,7.85,9.3,6.76,7.45,9.4,8.65,7.95,9.8,7.97,6.95,8.3,8.3,8.45,7.2,8.0,7.04,9.55,8.3,8.7,8.0,7.1,8.68,8.5,8.29,7.0,7.8,8.45,8.0,7.04,7.8,8.3,8.7,8.0,7.1,8.68,8.5,8.29,9.35,7.8,7.0,7.15,8.28,8.75,8.5,08/09/2019,105997.8


In [0]:
df_train_h_dummy_stock_sx_rate_gold.drop(columns=['Gold_date'],inplace=True)
df_test_h_dummy_stock_sx_rate_gold.drop(columns=['Gold_date'],inplace=True)

In [1527]:
df_train_h_dummy_stock_sx_rate_gold.sample(5)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close,Allahabad Bank,Andhra Bank,Bank of Baroda,Bank of India,Bank of Maharashtra,Canara Bank,Central Bank of India,Corporation Bank,Indian Bank,Indian Overseas Bank,Oriental Bank of Commerce,Punjab and Sind Bank,Punjab National Bank,State Bank of India,Syndicate Bank,UCO Bank,Union Bank of India,United Bank of India,Axis Bank Ltd.,Bandhan Bank Ltd.,Catholic Syrian Bank Ltd.,City Union Bank Ltd.,Development Credit Bank Ltd.,Dhanalaxmi Bank Ltd.,Federal Bank Ltd.,HDFC Bank Ltd.,ICICI Bank Ltd.,IDFC Bank Ltd,Indusind Bank,Jammu and Kashmir Bank Ltd.,Karnataka Bank Ltd.,Karur Vysya Bank Ltd.,...,Nainital Bank,RBL Bank,South Indian Bank Ltd.,Tamilnad Mercantile Bank Ltd.,Yes Bank Ltd.,AB Bank Ltd.,Abu Dhabi Commercial Bank Ltd.,American Express Banking Corporation,Australia and New zealand banking group Ltd.,Bank International Indonesia,Bank of Bahrain and Kuwait,Bank of America,Bank of Ceylon,Bank of Nova Scotia,Bank of Tokyo Mits UFJ Ltd.,Barclays Bank,BNP Paribas,Citi Bank,Credit Agricole Corporate & Investment Bank,Credit Suisse AG Bank,CTBC Bank Co.Ltd,Deutsche Bank,Development Bank of Singapore,Doha Bank QSC,Emirates NBD Bank (P.J.S.C),First Abu Dhabi Bank PJSC,Firstrand Bank Ltd,Hongkong & Shanghai Bkg. Corpn,Industrial and Commercial Bank of China,Industrial Bank of Korea,JP Morgan Chase Bank,KEB Hana Bank,Krung Thai Bank PCL,Mashreq Bank,Mizuho Corporate Bank,National Australia Bank,Qatar National Bank S.A.Q,Rabobank International,Sber Bank,Shinhan Bank,Societe Generale,Sonali Bank Ltd.,Standard Chartered Bank,State Bank of Mauritius,Sumitomo Mitsui Banking Corporation,The Royal Bank of Scotland,United Overseas Bank,Westpac Banking Corporation,Woori Bank,Gold_price
6445,09/20/2018,6,3005,0,20,2018,9,4,146.1,147.565501,1372269.0,6339.0,202499563.0,412706.0,30.07,7.70,-5.90,37121.22,8.50,8.7,8.55,8.6,8.75,8.6,8.6,8.95,8.60,8.70,8.65,8.80,8.45,8.45,8.80,8.65,8.65,8.85,8.75,10.50,9.9,9.30,10.62,9.65,9.20,8.65,8.55,9.20,9.65,8.85,8.95,9.55,...,8.40,9.90,9.35,9.10,9.70,6.71,9.20,6.42,8.40,9.60,8.80,7.60,9.85,8.67,7.80,8.5,9.55,8.45,8.54,8.25,8.89,10.10,8.90,8.80,8.90,6.7,8.87,8.90,8.94,7.2,8.90,8.30,8.00,8.95,8.70,8.4,8.05,8.55,8.35,8.35,8.65,6.15,9.15,9.20,8.70,6.65,9.54,8.75,8.50,87635.6
153,09/01/2017,0,66,0,1,2017,9,0,198.7,200.110072,1136943.0,6591.0,227513745.0,266898.0,23.48,4.80,-2.30,31892.23,8.45,8.4,8.35,8.3,8.70,8.3,8.3,8.65,8.35,8.55,8.40,8.55,8.15,8.00,8.50,8.45,8.20,8.75,8.25,10.30,9.8,9.25,9.72,9.80,8.90,8.15,8.20,8.60,8.95,8.90,8.85,9.00,...,8.30,9.30,9.00,9.40,8.80,8.08,8.25,6.89,7.20,9.00,8.05,7.55,9.90,8.20,7.10,7.5,8.15,8.05,7.54,7.41,7.86,9.20,8.20,8.05,8.05,6.9,8.26,8.07,8.50,8.4,8.20,8.25,8.25,7.45,7.30,7.3,7.30,7.65,8.87,8.00,7.75,8.27,8.55,8.75,7.35,8.70,7.99,8.50,7.50,84535.1
11295,02/18/2018,13,6,0,18,2018,2,3,160.5,163.593331,442436.0,2231.0,72379579.0,123723.0,27.96,9.40,-6.75,34010.76,8.25,8.4,8.30,8.3,8.65,8.4,8.3,8.75,8.25,8.40,8.50,8.40,8.30,8.15,8.45,8.45,8.20,8.60,8.40,9.91,9.8,9.05,9.87,9.65,8.90,8.30,8.30,8.75,9.25,8.95,8.75,9.16,...,8.50,9.45,9.00,8.95,9.25,6.48,9.25,7.76,8.10,8.85,8.05,7.90,9.79,7.35,7.15,8.1,8.60,8.00,7.93,7.24,8.40,9.35,8.65,8.40,8.00,6.9,8.60,8.30,8.34,5.8,8.95,8.25,8.00,7.55,7.55,7.9,7.65,7.55,8.52,8.15,8.30,7.18,8.65,8.95,7.55,6.75,8.15,8.50,7.70,86424.8
7792,10/14/2017,8,798,0,14,2017,10,2,204.3,204.779006,694276.0,3665.0,142173149.0,126406.0,18.21,4.40,1.20,32432.69,8.30,8.4,8.30,8.3,8.65,8.3,8.3,8.65,8.35,8.40,8.40,8.40,8.15,7.95,8.45,8.45,8.20,8.60,8.25,10.20,9.8,9.25,9.72,9.80,8.90,8.10,8.20,8.50,8.85,8.80,8.75,9.00,...,8.35,9.20,9.00,9.15,8.80,7.26,8.20,7.32,7.50,9.00,8.40,7.75,9.95,7.09,7.05,7.5,8.00,8.10,7.32,7.07,8.38,9.25,8.30,8.35,8.05,7.0,8.27,8.05,8.01,6.5,8.35,8.25,8.25,7.55,7.30,7.2,7.45,7.55,8.87,8.00,7.75,7.04,8.55,8.75,7.20,8.60,

In [1528]:
df_test_h_dummy_stock_sx_rate_gold.sample(6)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close,Allahabad Bank,Andhra Bank,Bank of Baroda,Bank of India,Bank of Maharashtra,Canara Bank,Central Bank of India,Corporation Bank,Indian Bank,Indian Overseas Bank,Oriental Bank of Commerce,Punjab and Sind Bank,Punjab National Bank,State Bank of India,Syndicate Bank,UCO Bank,Union Bank of India,United Bank of India,Axis Bank Ltd.,Bandhan Bank Ltd.,Catholic Syrian Bank Ltd.,City Union Bank Ltd.,Development Credit Bank Ltd.,Dhanalaxmi Bank Ltd.,Federal Bank Ltd.,HDFC Bank Ltd.,ICICI Bank Ltd.,IDFC Bank Ltd,Indusind Bank,Jammu and Kashmir Bank Ltd.,Karnataka Bank Ltd.,Karur Vysya Bank Ltd.,Kotak Mahindra Bank,...,Nainital Bank,RBL Bank,South Indian Bank Ltd.,Tamilnad Mercantile Bank Ltd.,Yes Bank Ltd.,AB Bank Ltd.,Abu Dhabi Commercial Bank Ltd.,American Express Banking Corporation,Australia and New zealand banking group Ltd.,Bank International Indonesia,Bank of Bahrain and Kuwait,Bank of America,Bank of Ceylon,Bank of Nova Scotia,Bank of Tokyo Mits UFJ Ltd.,Barclays Bank,BNP Paribas,Citi Bank,Credit Agricole Corporate & Investment Bank,Credit Suisse AG Bank,CTBC Bank Co.Ltd,Deutsche Bank,Development Bank of Singapore,Doha Bank QSC,Emirates NBD Bank (P.J.S.C),First Abu Dhabi Bank PJSC,Firstrand Bank Ltd,Hongkong & Shanghai Bkg. Corpn,Industrial and Commercial Bank of China,Industrial Bank of Korea,JP Morgan Chase Bank,KEB Hana Bank,Krung Thai Bank PCL,Mashreq Bank,Mizuho Corporate Bank,National Australia Bank,Qatar National Bank S.A.Q,Rabobank International,Sber Bank,Shinhan Bank,Societe Generale,Sonali Bank Ltd.,Standard Chartered Bank,State Bank of Mauritius,Sumitomo Mitsui Banking Corporation,The Royal Bank of Scotland,United Overseas Bank,Westpac Banking Corporation,Woori Bank,Gold_price
1306,07/28/2019,14,0,28,2019,7,3,103.80,104.103909,801348.0,3943.0,83423459.0,104582.0,13.05,4.25,1.80,37686.37,8.40,8.45,8.45,8.35,8.5,8.5,8.30,8.70,8.60,8.5,8.55,8.5,8.30,8.25,8.35,8.5,8.50,8.55,8.55,10.11,9.9,9.15,10.65,9.9,9.17,8.60,8.65,9.50,9.65,8.75,9.35,9.55,8.75,...,8.4,9.95,9.45,9.2,9.75,7.72,9.55,7.03,8.05,9.75,8.75,8.25,9.87,8.41,7.40,8.4,8.05,8.50,7.71,8.00,7.65,9.70,8.3,8.75,8.2,7.4,8.77,8.65,8.08,5.9,8.05,8.45,7.75,8.17,7.85,9.3,8.1,8.65,8.34,8.45,7.90,5.82,9.45,9.45,6.9,7.05,8.41,8.75,8.55,97538.4
97,07/28/2019,1,0,28,2019,7,3,103.80,104.103909,801348.0,3943.0,83423459.0,104582.0,13.05,4.25,1.80,37686.37,8.40,8.45,8.45,8.35,8.5,8.5,8.30,8.70,8.60,8.5,8.55,8.5,8.30,8.25,8.35,8.5,8.50,8.55,8.55,10.11,9.9,9.15,10.65,9.9,9.17,8.60,8.65,9.50,9.65,8.75,9.35,9.55,8.75,...,8.4,9.95,9.45,9.2,9.75,7.72,9.55,7.03,8.05,9.75,8.75,8.25,9.87,8.41,7.40,8.4,8.05,8.50,7.71,8.00,7.65,9.70,8.3,8.75,8.2,7.4,8.77,8.65,8.08,5.9,8.05,8.45,7.75,8.17,7.85,9.3,8.1,8.65,8.34,8.45,7.90,5.82,9.45,9.45,6.9,7.05,8.41,8.75,8.55,97538.4
768,08/17/2019,9,0,17,2019,8,2,105.00,104.220677,590556.0,2471.0,61548146.0,66144.0,11.20,3.55,1.20,37402.49,8.40,8.40,8.40,8.35,8.5,8.4,8.25,8.65,8.45,8.5,8.40,8.5,8.30,8.15,8.35,8.5,8.35,8.45,8.45,10.11,9.9,9.00,10.65,9.9,8.98,8.45,8.55,9.30,9.55,8.70,9.25,9.45,8.75,...,8.4,9.95,9.25,9.1,9.70,7.85,9.30,6.76,7.45,9.40,8.65,7.95,9.80,7.97,6.95,8.3,8.30,8.45,7.20,8.00,7.04,9.55,8.3,8.70,8.0,7.1,8.68,8.50,8.29,7.0,7.80,8.45,8.00,7.04,7.80,8.3,8.7,8.00,7.10,8.68,8.50,8.29,9.35,7.80,7.0,7.15,8.28,8.75,8.50,106890.0
15,08/08/2019,0,0,8,2019,8,4,99.45,97.882669,822420.0,3322.0,80500665.0,104562.0,12.71,3.95,3.05,37327.36,8.40,8.40,8.40,8.35,8.5,8.4,8.25,8.65,8.45,8.5,8.40,8.5,8.30,8.15,8.35,8.5,8.35,8.45,8.45,10.11,9.9,9.00,10.65,9.9,8.98,8.45,8.55,9.30,9.55,8.70,9.25,9.45,8.75,...,8.4,9.95,9.25,9.1,9.70,7.85,9.30,6.76,7.45,9.40,8.65,7.95,9.80,7.97,6.95,8.3,8.30,8.45,7.20,8.00,7.04,9.55,8.3,8.70,8.0,7.1,8.68,8.50,8.29,7.0,7.80,8.45,8.00,7.04,7.80,8.3,8.7,8.00,7.10,8.68,8.50,8.29,9.35,7.80,7.0,7.15,8.28,8.75,8.50,105742.1
11

In [1529]:
print(df_train_h_dummy_stock_sx_rate_gold.shape,"| ",df_test_h_dummy_stock_sx_rate_gold.shape)

(13504, 102) |  (1488, 101)


In [0]:
####################################################################### Adding the Nifty50-2017-19 ###################################################################

In [0]:
df_N50 =  pd.read_csv('/content/drive/My Drive/Hackathon/LTFS/Nifty50-2017-19.csv')

In [1532]:
df_N50.head(3)

,nift50_Date,nift50_Open,nift50_High,nift50_Low,nift50_Close,nift50_Shares_Traded,nift50_Turnover (Rs. Cr)
0,3-Apr-17,9220.6,9245.35,9192.40,9237.85,143444541,8337.06
1,5-Apr-17,9264.4,9273.90,9215.40,9265.15,190529461,11461.96
2,6-Apr-17,9245.8,9267.95,9218.85,9261.95,191937207,10332.60


In [0]:

df_N50['nift50_Date'] = df_N50['nift50_Date'].astype('datetime64[ns]')


In [0]:
df_N50['nift50_Date'] = df_N50.nift50_Date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [1535]:
df_N50.sample(4)

,nift50_Date,nift50_Open,nift50_High,nift50_Low,nift50_Close,nift50_Shares_Traded,nift50_Turnover (Rs. Cr)
375,10/08/2018,10310.15,10398.35,10198.40,10348.05,470279031,22130.94
371,10/01/2018,10930.90,11035.65,10821.55,11008.30,398987274,20391.72
609,09/20/2019,10746.80,11381.90,10691.00,11274.20,1356767450,54081.53
451,01/28/2019,10792.45,10804.45,10630.95,10661.55,419682627,21144.33


In [0]:

df_train_h_dummy_stock_sx_rate_gold_N50 = pd.merge(df_train_h_dummy_stock_sx_rate_gold,df_N50,how='left',left_on='application_date',right_on='nift50_Date')

In [0]:
df_test_h_dummy_stock_sx_rate_gold_N50= pd.merge(df_test_h_dummy_stock_sx_rate_gold,df_N50,how='left',left_on='application_date',right_on='nift50_Date')

In [1538]:
print(df_train_h_dummy_stock_sx_rate_gold_N50.shape," | ", df_test_h_dummy_stock_sx_rate_gold_N50.shape)

(13504, 109)  |  (1488, 108)


In [1539]:
df_train_h_dummy_stock_sx_rate_gold_N50.sample(4)

,application_date,state,case_count,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close,Allahabad Bank,Andhra Bank,Bank of Baroda,Bank of India,Bank of Maharashtra,Canara Bank,Central Bank of India,Corporation Bank,Indian Bank,Indian Overseas Bank,Oriental Bank of Commerce,Punjab and Sind Bank,Punjab National Bank,State Bank of India,Syndicate Bank,UCO Bank,Union Bank of India,United Bank of India,Axis Bank Ltd.,Bandhan Bank Ltd.,Catholic Syrian Bank Ltd.,City Union Bank Ltd.,Development Credit Bank Ltd.,Dhanalaxmi Bank Ltd.,Federal Bank Ltd.,HDFC Bank Ltd.,ICICI Bank Ltd.,IDFC Bank Ltd,Indusind Bank,Jammu and Kashmir Bank Ltd.,Karnataka Bank Ltd.,Karur Vysya Bank Ltd.,...,American Express Banking Corporation,Australia and New zealand banking group Ltd.,Bank International Indonesia,Bank of Bahrain and Kuwait,Bank of America,Bank of Ceylon,Bank of Nova Scotia,Bank of Tokyo Mits UFJ Ltd.,Barclays Bank,BNP Paribas,Citi Bank,Credit Agricole Corporate & Investment Bank,Credit Suisse AG Bank,CTBC Bank Co.Ltd,Deutsche Bank,Development Bank of Singapore,Doha Bank QSC,Emirates NBD Bank (P.J.S.C),First Abu Dhabi Bank PJSC,Firstrand Bank Ltd,Hongkong & Shanghai Bkg. Corpn,Industrial and Commercial Bank of China,Industrial Bank of Korea,JP Morgan Chase Bank,KEB Hana Bank,Krung Thai Bank PCL,Mashreq Bank,Mizuho Corporate Bank,National Australia Bank,Qatar National Bank S.A.Q,Rabobank International,Sber Bank,Shinhan Bank,Societe Generale,Sonali Bank Ltd.,Standard Chartered Bank,State Bank of Mauritius,Sumitomo Mitsui Banking Corporation,The Royal Bank of Scotland,United Overseas Bank,Westpac Banking Corporation,Woori Bank,Gold_price,nift50_Date,nift50_Open,nift50_High,nift50_Low,nift50_Close,nift50_Shares_Traded,nift50_Turnover (Rs. Cr)
9953,01/30/2019,11,0,0,30,2019,1,6,132.50,131.495869,547790.0,2613.0,72032122.0,84571.0,15.44,3.90,-0.10,35591.25,8.75,8.75,8.75,8.75,8.75,8.7,8.65,8.95,8.75,8.80,8.75,8.85,8.55,8.55,8.75,8.70,8.70,8.85,8.9,10.45,9.9,9.30,10.84,9.90,9.20,8.75,8.80,9.25,9.90,9.00,9.15,9.65,...,7.22,8.25,9.85,8.65,7.8,9.87,8.85,7.90,8.90,9.50,8.80,8.17,8.30,7.70,10.30,8.85,9.15,8.7,7.3,9.15,9.15,9.05,5.9,9.00,8.45,8.25,8.90,8.30,9.3,8.10,8.50,8.52,8.60,8.45,7.02,9.60,9.80,8.35,6.50,9.04,8.75,8.70,93262.9,01/30/2019,10702.25,10710.2,10612.85,10651.8,410107910.0,21214.81
2452,05/05/2019,2,49,0,5,2019,5,3,127.25,127.706118,664400.0,4303.0,84847945.0,196650.0,29.60,4.95,-2.85,38963.26,8.55,8.75,8.70,8.65,8.60,8.7,8.55,8.85,8.65,8.65,8.70,8.75,8.45,8.45,8.65,8.65,8.60,8.75,8.7,10.11,9.9,9.30,10.84,9.90,9.15,8.70,8.75,9.50,9.75,8.85,9.40,9.65,...,7.25,8.05,9.75,8.90,8.4,9.78,8.41,7.50,8.45,8.65,8.70,7.78,8.08,7.95,10.10,8.25,8.95,8.5,7.9,9.05,8.78,8.54,6.9,8.85,8.45,8.25,8.55,8.15,9.3,8.30,8.55,8.73,8.60,8.20,5.69,9.45,10.00,7.55,6.20,8.67,8.75,8.65,88730.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9002,10/15/2018,10,1513,0,15,2018,10,1,130.75,130.722440,873520.0,11936.0,114188666.0,164043.0,18.78,6.25,-2.75,34865.10,8.55,8.70,8.65,8.65,8.75,8.7,8.60,8.95,8.70,8.75,8.75,8.80,8.50,8.50,8.80,8.65,8.65,8.85,8.8,10.55,9.9,9.30,10.74,9.65,9.20,8.70,8.70,9.20,9.75,8.90,8.95,9.65,...,6.42,8.40,9.70,8.80,7.5,9.87,8.67,7.75,9.25,9.30,8.45,8.55,7.55,8.09,10.10,8.90,8.85,8.8,6.7,8.92,9.14,8.89,7.2,9.15,8.45,8.00,8.95,8.65,8.4,8.25,8.45,8.35,8.35,8.75,7.19,9.25,9.40,8.65,6.50,9.60,8.75,8.50,90810.3,10/15/2018,10524.20,10526.3,10410.15,10512.5,267517728.0,14372.15
1972,01/10/2018,2,583,0,10,2018,1,6,181.10,182.377621,426409.0,2144.0,77767459.0,152576.0,35.78,5.15,-2.15,34433.07,8.25,8.40,8.30,8.30,8.65,8.3,8.30,8.65,8.25,8.40,8.35,8.40,8.15,7.95,8.45,8.45,8.20,8.60,8.4,9.95,9.8,9.25,9.87,9.65,8.90,8.20,8.20,8.60,9.10,8.80,8.75,9.02,...,7.37,7.80,9.05,7.85,7.9,9.91,7.35,7.15,7.70,8.00,8.00,7.60,7.10,8.06,9.25,8.65,8.25,8.1,6.6,8.48,8.15,8.07,6.2,9.10,8.25,8.00,7.60,7.75,7.6,8.35,7.40,8.52,8.00,7.95,6.76,8.55,8.75,7.50,7.

In [1540]:
df_test_h_dummy_stock_sx_rate_gold_N50.sample(4)

,application_date,state,Holiday_Flag,day,Year,Month,Weekday_Name,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Deliverable Quantity,% Deli. Qty to Traded Qty,Spread High-Low,Spread Close-Open,sx_Close,Allahabad Bank,Andhra Bank,Bank of Baroda,Bank of India,Bank of Maharashtra,Canara Bank,Central Bank of India,Corporation Bank,Indian Bank,Indian Overseas Bank,Oriental Bank of Commerce,Punjab and Sind Bank,Punjab National Bank,State Bank of India,Syndicate Bank,UCO Bank,Union Bank of India,United Bank of India,Axis Bank Ltd.,Bandhan Bank Ltd.,Catholic Syrian Bank Ltd.,City Union Bank Ltd.,Development Credit Bank Ltd.,Dhanalaxmi Bank Ltd.,Federal Bank Ltd.,HDFC Bank Ltd.,ICICI Bank Ltd.,IDFC Bank Ltd,Indusind Bank,Jammu and Kashmir Bank Ltd.,Karnataka Bank Ltd.,Karur Vysya Bank Ltd.,Kotak Mahindra Bank,...,American Express Banking Corporation,Australia and New zealand banking group Ltd.,Bank International Indonesia,Bank of Bahrain and Kuwait,Bank of America,Bank of Ceylon,Bank of Nova Scotia,Bank of Tokyo Mits UFJ Ltd.,Barclays Bank,BNP Paribas,Citi Bank,Credit Agricole Corporate & Investment Bank,Credit Suisse AG Bank,CTBC Bank Co.Ltd,Deutsche Bank,Development Bank of Singapore,Doha Bank QSC,Emirates NBD Bank (P.J.S.C),First Abu Dhabi Bank PJSC,Firstrand Bank Ltd,Hongkong & Shanghai Bkg. Corpn,Industrial and Commercial Bank of China,Industrial Bank of Korea,JP Morgan Chase Bank,KEB Hana Bank,Krung Thai Bank PCL,Mashreq Bank,Mizuho Corporate Bank,National Australia Bank,Qatar National Bank S.A.Q,Rabobank International,Sber Bank,Shinhan Bank,Societe Generale,Sonali Bank Ltd.,Standard Chartered Bank,State Bank of Mauritius,Sumitomo Mitsui Banking Corporation,The Royal Bank of Scotland,United Overseas Bank,Westpac Banking Corporation,Woori Bank,Gold_price,nift50_Date,nift50_Open,nift50_High,nift50_Low,nift50_Close,nift50_Shares_Traded,nift50_Turnover (Rs. Cr)
495,08/23/2019,5,0,23,2019,8,0,89.90,90.107863,1541446.0,7129.0,138896405.0,260097.0,16.87,4.80,-3.1,36701.16,8.40,8.4,8.40,8.35,8.5,8.4,8.25,8.65,8.45,8.5,8.40,8.5,8.30,8.15,8.35,8.5,8.35,8.45,8.45,10.11,9.9,9.0,10.65,9.9,8.98,8.45,8.55,9.30,9.55,8.70,9.25,9.45,8.75,...,6.76,7.45,9.4,8.65,7.95,9.80,7.97,6.95,8.3,8.30,8.45,7.20,8.00,7.04,9.55,8.3,8.70,8.0,7.1,8.68,8.50,8.29,7.0,7.8,8.45,8.00,7.04,7.80,8.3,8.7,8.0,7.1,8.68,8.50,8.29,9.35,7.8,7.0,7.15,8.28,8.75,8.5,107766.0,08/23/2019,10699.6,10862.55,10637.15,10829.35,667079625.0,20983.75
1477,10/14/2019,15,0,14,2019,10,1,81.65,81.920895,476594.0,2351.0,39043007.0,84100.0,17.65,2.85,0.6,38214.47,8.35,8.3,8.35,8.30,8.4,8.4,8.20,8.60,8.35,8.5,8.35,8.5,8.25,8.05,8.35,8.4,8.30,8.45,8.35,10.06,9.9,9.0,10.47,9.8,8.90,8.35,8.45,9.35,9.45,8.65,9.25,9.45,8.60,...,6.98,7.45,9.4,8.60,7.15,9.74,8.14,7.00,8.0,7.75,8.30,7.12,7.33,7.16,9.50,8.3,8.55,8.1,7.0,8.46,8.45,7.40,6.3,7.7,8.35,8.00,7.99,7.45,8.3,7.5,8.1,7.1,8.30,7.65,7.95,9.25,7.8,6.6,7.15,7.63,8.75,8.4,106143.7,10/14/2019,11335.9,11420.45,11290.05,11341.15,587579191.0,17499.48
1362,09/22/2019,14,0,22,2019,9,3,99.10,96.315050,2023984.0,7954.0,194940120.0,452106.0,22.34,10.25,7.2,39090.03,8.35,8.3,8.35,8.30,8.4,8.4,8.20,8.60,8.35,8.5,8.35,8.5,8.25,8.05,8.35,8.4,8.30,8.45,8.35,10.06,9.9,9.0,10.47,9.8,8.90,8.35,8.45,9.35,9.45,8.65,9.25,9.45,8.60,...,6.98,7.45,9.4,8.60,7.15,9.74,8.14,7.00,8.0,7.75,8.30,7.12,7.33,7.16,9.50,8.3,8.55,8.1,7.0,8.46,8.45,7.40,6.3,7.7,8.35,7.75,7.99,7.45,8.3,7.5,8.1,7.1,8.30,7.65,7.95,9.25,7.8,6.6,7.15,7.63,8.75,8.4,107971.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,08/06/2019,7,0,6,2019,8,5,98.70,97.433789,872804.0,3425.0,85040601.0,107726.0,12.34,5.45,4.6,36976.85,8.40,8.4,8.40,8.35,8.5,8.4,8.25,8.65,8.45,8.5,8.40,8.5,8.30,8.15,8.35,8.5,8.35,8.45,8.45,10.11,9.9,9.0,10.65,9.9,8.98,8.45,8.55,9.30,9.55,8.70,9.25,9.45,8.75,...,6.76,7.45,9.4,8.65,7.95,9.80,7.97,6.95,8.3,8.30,8.45,7.20,8.00,7.04,9.55,8.3,8.70,8.0,7.1,8.68,8.50,8.29,7.0,7.8,8.45,8.00,7.04,7.80,8.3,8.7,8.0,7.1,8.68,8.50,8.29,9.35,7.8,7.0,7.15,8.28,8.75,8.5,103776.3,08/06/2019,10815.4,11018.55,1

In [0]:
df_train_h_dummy_stock_sx_rate_gold_N50.drop(columns='nift50_Date',inplace=True)
df_test_h_dummy_stock_sx_rate_gold_N50.drop(columns='nift50_Date',inplace=True)

In [1542]:
print(df_train_h_dummy_stock_sx_rate_gold_N50.isnull().sum().sum()," | ", df_test_h_dummy_stock_sx_rate_gold_N50.isnull().sum().sum())

26208  |  3168


In [0]:
df_train_h_dummy_stock_sx_rate_gold_N50.fillna(method='ffill',inplace=True)
df_test_h_dummy_stock_sx_rate_gold_N50.fillna(method='bfill',inplace=True)
############################################################
df_train_h_dummy_stock_sx_rate_gold_N50.fillna(method='bfill',inplace=True)
df_test_h_dummy_stock_sx_rate_gold_N50.fillna(method='ffill',inplace=True)

In [1544]:
print(df_train_h_dummy_stock_sx_rate_gold_N50.isnull().sum().sum()," | ", df_test_h_dummy_stock_sx_rate_gold_N50.isnull().sum().sum())

0  |  0


In [1545]:
print(df_train_h_dummy_stock_sx_rate_gold_N50.shape," | ", df_test_h_dummy_stock_sx_rate_gold_N50.shape)

(13504, 108)  |  (1488, 107)


In [0]:
######################################################################## Feature Selection ############################################################

In [0]:
# pearson's correlation feature selection for numeric input and numeric output
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

#apply SelectKBest class to extract top 10 best features # df_train_h_dummy_stock_sx_rate_gold_N50
bestfeatures = SelectKBest(score_func=f_regression, k=10)
fit = bestfeatures.fit(df_train_h_dummy_stock_sx_rate_gold_N50.drop(columns=['application_date','case_count']),df_train_h_dummy_stock_sx_rate_gold_N50['case_count'])
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(df_train_h_dummy_stock_sx_rate_gold_N50.drop(columns=['application_date','case_count']).columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
#print(featureScores.nlargest(50,'Score'))  #print 10 best features

In [0]:
featureScores.sort_values(by='Score',ascending=False,inplace=True)

In [0]:
top_features_regr_60  = featureScores.head(30)

In [1550]:
top_features_regr_60_col = top_features_regr_60['Specs']
top_features_regr_60_col

0                                             state
95                       The Royal Bank of Scotland
88                                        Sber Bank
2                                               day
54                                    Yes Bank Ltd.
15                                         sx_Close
101                                     nift50_High
90                                 Societe Generale
103                                    nift50_Close
100                                     nift50_Open
83                                    Mashreq  Bank
102                                      nift50_Low
77                   Hongkong & Shanghai Bkg. Corpn
72                    Development Bank of Singapore
34                                   Axis Bank Ltd.
58     Australia and New zealand banking group Ltd.
86                        Qatar National Bank S.A.Q
3                                              Year
99                                       Gold_price
71          

In [0]:
#################################################################  *********  CREATING THE MODEL ********* ####################################################################################

In [0]:
X = df_train_h_dummy_stock_sx_rate_gold_N50[top_features_regr_60_col]

In [0]:
Y = df_train_h_dummy_stock_sx_rate_gold_N50['case_count']

In [0]:
X_test = df_test_h_dummy_stock_sx_rate_gold_N50[top_features_regr_60_col]

In [0]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import train_test_split,cross_val_score
import io
from datetime import datetime
import time

In [0]:
X_train,X_valid,y_train,y_valid = train_test_split(X,Y, test_size = 0.3,shuffle = True ,random_state=10)

In [1557]:
print('X_train shape -',X_train.shape)
print('y_train shape -',y_train.shape)
print('X_valid shape -',X_valid.shape)
print('y_valid shape -',y_valid.shape)

X_train shape - (9452, 30)
y_train shape - (9452,)
X_valid shape - (4052, 30)
y_valid shape - (4052,)


In [0]:
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()    #  MinMaxScaler() #StandardScaler
X_train_minmax = mm_scaler.fit_transform(X_train)
X_valid_minmax = mm_scaler.transform(X_valid)
X_test_minmax = mm_scaler.transform(X_test)

In [0]:
 #Machine Learning Algorithm (MLA) Selection and Initialization
  
  MLA =  [ #Ensemble Methods 
          ensemble.AdaBoostRegressor()  ,
          ensemble.BaggingRegressor(),                                    
          ensemble.ExtraTreesRegressor(),                                   
          ensemble.GradientBoostingRegressor(),                         
          ensemble.RandomForestRegressor(), 
            
          linear_model.LinearRegression(),
          #GLM                                                           
          #linear_model.LogisticRegressionCV(),
          #SVM  
          svm.LinearSVC(),                                          

          #Trees  
          tree.DecisionTreeRegressor(),
          ensemble.AdaBoostRegressor(),      
          XGBClassifier()  

  ]

In [0]:
MLA_columns = ['MLA_Name']
MLA_compare = pd.DataFrame(columns = MLA_columns)


In [0]:
rf = ensemble.RandomForestRegressor()  # alg    # ensemble.AdaBoostRegressor()   ## tree.DecisionTreeRegressor() ,   ensemble.RandomForestRegressor() ,  svm.SVR() , ensemble.GradientBoostingRegressor()

In [1562]:
rf.__class__.__name__

'RandomForestRegressor'

In [1563]:
rf.fit(X_train_minmax,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [0]:
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100

In [1565]:
mean_absolute_percentage_error(y_valid,rf.predict(X_valid_minmax))

55.262024514589115

In [1566]:
metrics.mean_absolute_error(y_valid,rf.predict(X_valid_minmax))

220.5603232971372

In [1567]:
metrics.r2_score(y_valid,rf.predict(X_valid_minmax), sample_weight=None, multioutput='uniform_average')

0.9000828103441811

# **SCORE -**

In [0]:

# Score = 0.5*(mean_absolute_percentage_error_segement1)+0.5*(mean_absolute_percentage_error_segement2)
# print(Score)

In [0]:
X_test_minmax = mm_scaler.transform(X_test)

In [1570]:
y_test_pred = rf.predict(X_test_minmax)
print(y_test_pred)

[1020.5   968.34  952.27 ... 2028.24 1900.1  1762.74]


In [1571]:
len(y_test_pred)

1488

In [1572]:
type(y_test_pred)

numpy.ndarray

In [1573]:
df_test_bkp.sample(5)

,application_date,state
369,10/22/2019,GUJARAT
922,10/17/2019,MADHYA PRADESH
378,7/30/2019,HARYANA
252,9/28/2019,CHHATTISGARH
539,10/6/2019,JHARKHAND


In [0]:
df_results = pd.concat([df_test_h_dummy_col,np.round(pd.Series(y_test_pred,name='case_count'),1).astype('int64')],axis=1,)

In [1575]:
df_results['case_count'].values

array([1020,  968,  952, ..., 2028, 1900, 1762])

In [0]:
df_result_nw = df_results[['application_date','case_count']].copy(deep=True)

In [1577]:
df_result_nw.shape

(1488, 2)

In [0]:
df_result_grp_sg2 =  df_result_nw.groupby(by='application_date').sum()

In [1579]:
df_result_grp_sg2.shape

(93, 1)

In [1580]:
df_result_grp_sg2.sample(6)

,case_count
application_date,
10/09/2019,7082
07/26/2019,21419
09/13/2019,27541
09/12/2019,25416
10/03/2019,5941
08/13/2019,26237


In [0]:
df_result_grp_sg2['segment'] =2

In [1582]:
df_result_grp_sg2.sample(6)

,case_count,segment
application_date,,
10/04/2019,5858,2
08/02/2019,5733,2
09/21/2019,25668,2
09/17/2019,28455,2
08/30/2019,5240,2
07/28/2019,15135,2


In [1583]:
df_result_grp_sg2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93 entries, 07/24/2019 to 10/24/2019
Data columns (total 2 columns):
case_count    93 non-null int64
segment       93 non-null int64
dtypes: int64(2)
memory usage: 2.2+ KB


In [0]:
df_result_grp_sg2.reset_index(inplace=True)

In [1585]:
df_result_grp_sg2

,application_date,case_count,segment
0,07/24/2019,22698,2
1,07/25/2019,22551,2
2,07/26/2019,21419,2
3,07/27/2019,19323,2
4,07/28/2019,15135,2
...,...,...,...
88,10/20/2019,28884,2
89,10/21/2019,26520,2
90,10/22/2019,26057,2
91,10/23/2019,23445,2


In [0]:
df_result_grp_sg2.sort_values(by='application_date',inplace=True)

In [0]:
import pytz
now = datetime.now()
tz = pytz.timezone('Asia/Kolkata')
timestamp = now.astimezone(tz)

In [0]:
from google.colab import files

df_result_grp_sg2.to_csv('/content/drive/My Drive/Hackathon/LTFS/25-JAN/Submission_Seg2/Submission_results_Segment-2_L&T Stocks_Bank Rate_Sensex_N50_' + str(rf.__class__.__name__) + '_' + str(timestamp)+ '.csv')

In [0]:
files.download('/content/drive/My Drive/Hackathon/LTFS/25-JAN/Submission_Seg2/Submission_results_Segment-2_L&T Stocks_Bank Rate_Sensex_N50_' + str(rf.__class__.__name__) + '_' + str(timestamp)+ '.csv')